In [2]:
# Basic Libraries

import numpy as np
import scipy as sp
import sympy as smp
from scipy.integrate import solve_ivp, odeint
import itertools
import mpmath
import opt_einsum as oe
import numba as nb
import symengine as sme
from scipy.spatial import KDTree
from scipy.spatial import ConvexHull
from scipy.spatial.distance import pdist

# For Plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# For Debugging and Profiling
import snoop
from cProfile import Profile
from pstats import SortKey, Stats
import pyperclip

In [3]:
def get_norm(T, g):
    """
    T should have confuguration either 'u' or 'uu
    Returned Configuration: Scalar
    """
    if T.ndim == 2:
        return np.einsum("ua,vb,ab,uv->", g, g, T, T)

    else:
        return np.einsum("ab,a,b->", g, T, T)

In [4]:
def levi_civita_symbol(dim=4):
    """
    Define Levi Cevita Tensor
    Config: uuuu
    """

    arr = np.zeros(tuple([dim for _ in range(dim)]))
    for x in itertools.permutations(tuple(range(dim))):
        mat = np.zeros((dim, dim), dtype=np.int32)
        for i, j in zip(range(dim), x):
            mat[i, j] = 1
        arr[x] = int(np.linalg.det(mat))
    return arr

In [27]:
# @snoop
def levi_civita_tensor(gk):
    #! uuuu
    epsu = levi_civita_symbol()
    det_g = smp.det(smp.Matrix(gk))
    return epsu / sme.sqrt(-(det_g))

In [6]:
def cov_metric_tensor(r, theta, a, M=1.0):
    """
    Define the metric tensor function.

    Parameters:
        a (float): Parameter 'a' in the metric.
        r (float): Parameter 'r' in the metric.
        theta (float): Parameter 'theta' in the metric.

    Returns:
        np.ndarray: The metric tensor at the given values of a, r, and theta.
        Configuration: ll
    """
    #! Correction Added Mass term in the symbols
    g = np.zeros((4, 4), dtype=type(r))

    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    g[0, 0] = -(1.0 - 2.0 * M * r / Sigma)
    g[1, 1] = Sigma / Delta
    g[2, 2] = Sigma
    g[3, 3] = (
        sme.sin(theta) ** 2
        * ((r**2 + a**2) ** 2 - Delta * a**2 * sme.sin(theta) ** 2)
        / Sigma
    )
    g[0, 3] = -2.0 * a * M * r * sme.sin(theta) ** 2 / Sigma
    g[3, 0] = g[0, 3]

    # return g.astype(float)
    return g

In [7]:
def cot_metric_tensor(r, theta, a, M=1.0):
    """
    Define the metric tensor function.

    Parameters:
        a (float): Parameter 'a' in the metric.
        r (float): Parameter 'r' in the metric.
        theta (float): Parameter 'theta' in the metric.

    Returns:
        np.ndarray: The metric tensor at the given values of a, r, and theta.
        Configuration: uu
    """
    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    denom = Delta * sme.sin(theta) ** 2

    g = np.zeros((4, 4), dtype=type(r))
    g[0, 0] = (
        -(
            sme.sin(theta) ** 2
            * ((r**2 + a**2) ** 2 - Delta * a**2 * sme.sin(theta) ** 2)
            / Sigma
        )
        / denom
    )

    g[1, 1] = Delta / Sigma

    g[2, 2] = 1.0 / Sigma

    # g[3, 3] = -(-(1.0 - 2.0 * r / Sigma)) / denom
    #! Corrected the g[3,3] term below
    g[3, 3] = (Delta - a**2 * sme.sin(theta) ** 2) / (denom * Sigma)

    g[0, 3] = (-2.0 * a * M * r * sme.sin(theta) ** 2 / Sigma) / denom

    g[3, 0] = g[0, 3]

    return g
    # return g

In [8]:
def kerr_christoffel(r, theta, a, M=1.0):
    """
    Function to give the christoffel symbols for kerr metric.
    The christoffel symbols are given as \Gamma ^i _{jk}

    From Reference Paper, Appendix
    Config: ull
    """

    cs = np.zeros((4, 4, 4), dtype=type(r))

    # Definitions
    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    cs[3, 0, 1] = M * (2 * r**2 - Sigma) / (Delta * Sigma**2) * a
    cs[0, 0, 1] = (
        M * (r**2 + a**2) * (r**2 - a**2 * sme.cos(theta) ** 2) / (Sigma**2 * Delta)
    )

    cs[3, 0, 2] = -2 * M * a * r * (sme.cos(theta) /sme.sin(theta)) / (Sigma**2)
    cs[0, 0, 2] = -2 * M * a**2 * r *sme.sin(theta) * sme.cos(theta) / Sigma**2

    cs[0, 1, 3] = (
        -M
        * a
        * (2 * r**2 * (r**2 + a**2) + Sigma * (r**2 - a**2))
        *sme.sin(theta) ** 2
        / (Delta * Sigma**2)
    )

    cs[3, 1, 3] = (
        r * Sigma * (Sigma - 2 * M * r)
        - M * a**2 * (2 * r**2 - Sigma) *sme.sin(theta) ** 2
    ) / (Delta * Sigma**2)

    cs[0, 2, 3] = M * a**3 * r *sme.sin(theta) ** 2 *sme.sin(2 * theta) / Sigma**2

    cs[3, 2, 3] = (
        (sme.cos(theta) /sme.sin(theta))
        * (Sigma**2 + 2 * M * a**2 * r *sme.sin(theta) ** 2)
        / Sigma**2
    )

    cs[1, 0, 0] = M * Delta * (2 * r**2 - Sigma) / Sigma**3

    cs[1, 0, 3] = -cs[1, 0, 0] * a *sme.sin(theta) ** 2

    cs[2, 0, 0] = -M * a * r *sme.sin(2 * theta) / Sigma**3 * a
    cs[2, 0, 3] = (
        2.0 * M * r * a * (r**2 + a**2) *sme.sin(theta) * sme.cos(theta) / Sigma**3
    )

    cs[1, 1, 1] = r / Sigma + (M - r) / Delta

    cs[1, 2, 2] = -r * Delta / Sigma
    cs[2, 1, 2] = -cs[1, 2, 2] / Delta

    cs[1, 1, 2] = cs[2, 2, 2] = -(a**2) *sme.sin(2 * theta) / (2 * Sigma)
    cs[2, 1, 1] = -cs[1, 1, 2] / Delta

    cs[1, 3, 3] = (
        -Delta
        * (r * Sigma**2 - M * a**2 * (2 * r**2 - Sigma) *sme.sin(theta) ** 2)
        *sme.sin(theta) ** 2
        / Sigma**3
    )

    cs[2, 3, 3] = (
        -(Delta * Sigma**2 + 2 * M * r * (r**2 + a**2) ** 2)
        *sme.sin(2 * theta)
        / (2 * Sigma**3)
    )

    cs[0, 1, 0] = cs[0, 0, 1]
    cs[0, 2, 0] = cs[0, 0, 2]
    cs[0, 3, 1] = cs[0, 1, 3]
    cs[0, 3, 2] = cs[0, 2, 3]

    cs[1, 3, 0] = cs[1, 0, 3]
    cs[1, 2, 1] = cs[1, 1, 2]

    cs[2, 3, 0] = cs[2, 0, 3]
    cs[2, 2, 1] = cs[2, 1, 2]

    cs[3, 1, 0] = cs[3, 0, 1]
    cs[3, 2, 0] = cs[3, 0, 2]
    cs[3, 3, 1] = cs[3, 1, 3]
    cs[3, 3, 2] = cs[3, 2, 3]

    # return symmetrize(arr=cs)+np.zeros((4,4,4))
    return cs + np.zeros((4, 4, 4))
    # return cs

In [9]:
def kerr_riemann_tensor(r, theta, a, M=1.0, config="ulll"):
    """
    Define variables

    Components of the Riemann tensor for Kerr Metric
    From Reference Paper, Appendix
    The Configuration is ulll
    """

    rijkl = np.zeros((4, 4, 4, 4), dtype=type(r))

    # Definitions
    Delta = a**2 - 2.0 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    rijkl[0, 0, 0, 3] = (
        2.0
        * M**2
        * sme.sin(theta) ** 2
        * a
        * r**2
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / Sigma**4
    )

    rijkl[0, 0, 1, 2] = (
        -2.0
        * M**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * r
        * sme.cos(theta)
        * a**2
        * sme.sin(theta)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 0, 1] = (
        M
        * r
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 0, 2] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2 * M * r)
        * a**2
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 1, 3] = (
        3.0
        * M
        * sme.sin(theta) ** 2
        * a
        * r
        * (r**2 + a**2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 2, 3] = (
        -M
        * sme.cos(theta)
        * sme.sin(theta)
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (2.0 * (r**2 + a**2) ** 2 + a**2 * sme.sin(theta) ** 2 * Delta)
        / (Sigma**3 * Delta)
    )

    # Antisymmetric under exchange of last two indices.
    rijkl[0, 0, 3, 0] = -rijkl[0, 0, 0, 3]
    rijkl[0, 0, 2, 1] = -rijkl[0, 0, 1, 2]
    rijkl[0, 1, 1, 0] = -rijkl[0, 1, 0, 1]
    rijkl[0, 1, 2, 0] = -rijkl[0, 1, 0, 2]
    rijkl[0, 1, 3, 1] = -rijkl[0, 1, 1, 3]
    rijkl[0, 1, 3, 2] = -rijkl[0, 1, 2, 3]

    #! -------------------------------------------------------------------------------------#

    rijkl[0, 2, 0, 1] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 4 * M * r)
        * a**2
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )
    rijkl[0, 2, 0, 2] = (
        -M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2 * a**2 * sme.sin(theta) ** 2)
        / Sigma**3
    )
    rijkl[0, 2, 1, 3] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * ((a**2 + r**2) ** 2 + 2.0 * a**2 * sme.sin(theta) ** 2 * Delta)
        * a
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )
    rijkl[0, 2, 2, 3] = (
        -3.0
        * M
        * sme.sin(theta) ** 2
        * r
        * a
        * (a**2 + r**2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / Sigma**3
    )
    rijkl[0, 3, 0, 3] = (
        -sme.sin(theta) ** 2
        * M
        * r
        * (r**2 + 3 * a**2 * sme.sin(theta) ** 2 - 3 * a**2)
        * ((a**2 + r**2) ** 2 - a**2 * sme.sin(theta) ** 2 * Delta)
        / Sigma**4
    )
    rijkl[0, 3, 1, 2] = (
        (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * M
        * ((a**2 + r**2) ** 2 - a**2 * sme.sin(theta) ** 2 * Delta)
        * a
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )

    # Symmetries
    rijkl[0, 2, 1, 0] = -rijkl[0, 2, 0, 1]
    rijkl[0, 2, 2, 0] = -rijkl[0, 2, 0, 2]
    rijkl[0, 2, 3, 1] = -rijkl[0, 2, 1, 3]
    rijkl[0, 2, 3, 2] = -rijkl[0, 2, 2, 3]
    rijkl[0, 3, 3, 0] = -rijkl[0, 3, 0, 3]
    rijkl[0, 3, 2, 1] = -rijkl[0, 3, 1, 2]

    #! ------------------------------------------------------------------------------#

    rijkl[1, 0, 0, 1] = (
        r
        * M
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (a**2 * sme.sin(theta) ** 2 + 2.0 * Delta)
        / Sigma**4
    )
    rijkl[1, 0, 0, 2] = (
        -3.0
        * M
        * Delta
        * a**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta)
        * sme.cos(theta)
        / Sigma**4
    )
    rijkl[1, 0, 1, 3] = (
        r
        * M
        * a
        * (3.0 * (r**2 + a**2) - 4.0 * M * r)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[1, 0, 2, 3] = (
        -a
        * M
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * Delta
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[1, 2, 0, 3] = (
        -sme.cos(theta)
        * M
        * sme.sin(theta)
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * Delta
        / Sigma**3
    )
    rijkl[1, 2, 1, 2] = -M * r * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / Sigma**2
    rijkl[1, 3, 0, 1] = (
        -sme.sin(theta) ** 2
        * M
        * r
        * a
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 4.0 * M * r)
        / Sigma**4
    )
    rijkl[1, 3, 0, 2] = (
        a
        * M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2.0 * a**2 * sme.sin(theta) ** 2)
        * Delta
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[1, 3, 1, 3] = (
        -sme.sin(theta) ** 2
        * M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * ((r**2 + a**2) ** 2 + 2.0 * a**2 * Delta * sme.sin(theta) ** 2)
        / Sigma**4
    )
    rijkl[1, 3, 2, 3] = (
        3.0
        * M
        * a**2
        * (r**2 + a**2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * Delta
        * sme.cos(theta)
        * sme.sin(theta) ** 3
        / Sigma**4
    )

    # Symmetries
    rijkl[1, 0, 1, 0] = -rijkl[1, 0, 0, 1]
    rijkl[1, 0, 2, 0] = -rijkl[1, 0, 0, 2]
    rijkl[1, 0, 3, 1] = -rijkl[1, 0, 1, 3]
    rijkl[1, 0, 3, 2] = -rijkl[1, 0, 2, 3]
    rijkl[1, 2, 3, 0] = -rijkl[1, 2, 0, 3]
    rijkl[1, 2, 2, 1] = -rijkl[1, 2, 1, 2]
    rijkl[1, 3, 1, 0] = -rijkl[1, 3, 0, 1]
    rijkl[1, 3, 2, 0] = -rijkl[1, 3, 0, 2]
    rijkl[1, 3, 3, 1] = -rijkl[1, 3, 1, 3]
    rijkl[1, 3, 3, 2] = -rijkl[1, 3, 2, 3]

    #! -------------------------------------------------------------------------------------#

    rijkl[2, 0, 0, 1] = (
        -3.0
        * M
        * a**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta)
        * sme.cos(theta)
        / Sigma**4
    )
    rijkl[2, 0, 0, 2] = (
        -M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (2.0 * a**2 * sme.sin(theta) ** 2 + Delta)
        / Sigma**4
    )
    rijkl[2, 0, 1, 3] = (
        -a
        * M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2.0 * a**2 * sme.sin(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[2, 0, 2, 3] = (
        -M
        * r
        * a
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2.0 * M * r)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[2, 1, 0, 3] = (
        M
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**3
    )
    rijkl[2, 1, 1, 2] = (
        M * r * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / (Delta * Sigma**2)
    )
    rijkl[2, 3, 0, 1] = (
        M
        * a
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[2, 3, 0, 2] = (
        M
        * r
        * a
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2.0 * M * r)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[2, 3, 1, 3] = (
        3.0
        * M
        * a**2
        * (r**2 + a**2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta) ** 3
        / Sigma**4
    )
    rijkl[2, 3, 2, 3] = (
        M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (2.0 * (r**2 + a**2) ** 2 + a**2 * Delta * sme.sin(theta) ** 2)
        * sme.sin(theta) ** 2
        / Sigma**4
    )

    # Symmetries
    rijkl[2, 0, 1, 0] = -rijkl[2, 0, 0, 1]
    rijkl[2, 0, 2, 0] = -rijkl[2, 0, 0, 2]
    rijkl[2, 0, 3, 1] = -rijkl[2, 0, 1, 3]
    rijkl[2, 0, 3, 2] = -rijkl[2, 0, 2, 3]
    rijkl[2, 1, 3, 0] = -rijkl[2, 1, 0, 3]
    rijkl[2, 1, 2, 1] = -rijkl[2, 1, 1, 2]
    rijkl[2, 3, 1, 0] = -rijkl[2, 3, 0, 1]
    rijkl[2, 3, 2, 0] = -rijkl[2, 3, 0, 2]
    # rijkl[2, 3, 2, 1] = -rijkl[2, 3, 1, 2]
    rijkl[2, 3, 3, 1] = -rijkl[2, 3, 1, 3]
    rijkl[2, 3, 3, 2] = -rijkl[2, 3, 2, 3]

    #! --------------------------------------------------------------------------#

    rijkl[3, 0, 1, 2] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (a**2 * sme.sin(theta) ** 2 - Delta)
        * a
        * (sme.cos(theta) / sme.sin(theta))
        / (Delta * Sigma**3)
    )
    rijkl[3, 1, 0, 1] = (
        3.0 * M * r * a * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / (Delta * Sigma**3)
    )
    rijkl[3, 1, 0, 2] = (
        -M
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (2.0 * a**2 * sme.sin(theta) ** 2 + Delta)
        * (sme.cos(theta) / sme.sin(theta))
        / (Delta * Sigma**3)
    )
    rijkl[3, 1, 1, 3] = (
        M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2.0 * a**2 * sme.sin(theta) ** 2)
        / (Delta * Sigma**3)
    )
    rijkl[3, 1, 2, 3] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2.0 * M * r)
        * a**2
        * sme.sin(theta)
        * sme.cos(theta)
        / (Delta * Sigma**3)
    )
    rijkl[3, 2, 0, 1] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (a**2 * sme.sin(theta) ** 2 + 2.0 * Delta)
        * a
        * (sme.cos(theta) / sme.sin(theta))
        / (Delta * Sigma**3)
    )
    rijkl[3, 2, 0, 2] = (
        -M * 3.0 * r * a * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / Sigma**3
    )
    rijkl[3, 2, 1, 3] = (
        -M
        * a**2
        * (3 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 4.0 * M * r)
        * sme.sin(theta)
        * sme.cos(theta)
        / (Delta * Sigma**3)
    )
    rijkl[3, 2, 2, 3] = (
        -M
        * r
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / Sigma**3
    )
    rijkl[3, 3, 0, 3] = (
        -2.0
        * M**2
        * a
        * r**2
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[3, 3, 1, 2] = (
        2.0
        * M**2
        * r
        * a**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / (Delta * Sigma**3)
    )

    # Symmetries
    rijkl[3, 0, 2, 1] = -rijkl[3, 0, 1, 2]
    rijkl[3, 1, 1, 0] = -rijkl[3, 1, 0, 1]
    rijkl[3, 1, 2, 0] = -rijkl[3, 1, 0, 2]
    rijkl[3, 1, 3, 1] = -rijkl[3, 1, 1, 3]
    rijkl[3, 1, 3, 2] = -rijkl[3, 1, 2, 3]
    rijkl[3, 2, 1, 0] = -rijkl[3, 2, 0, 1]
    rijkl[3, 2, 2, 0] = -rijkl[3, 2, 0, 2]
    rijkl[3, 2, 3, 1] = -rijkl[3, 2, 1, 3]
    rijkl[3, 2, 3, 2] = -rijkl[3, 2, 2, 3]
    rijkl[3, 3, 3, 0] = -rijkl[3, 3, 0, 3]
    rijkl[3, 3, 2, 1] = -rijkl[3, 3, 1, 2]

    #!---------------------------------------------------------------------#

    part1 = a**2 + 2 * r * (-2 * M + r) + a**2 * sme.cos(2 * theta)
    part2 = 3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta)
    part3 = (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4
    # Calculating Rtensor elements

    # rijkl[3, 0, 0, 3] =  (part1 + part2 + part3 + part4) / (Sigma**5)
    rijkl[3, 0, 0, 3] = 4 * M * r * part1 * part2 / part3
    rijkl[3, 0, 3, 0] = -rijkl[3, 0, 0, 3]

    # return rijkl
    if config == "ulll":
        return rijkl + np.zeros((4, 4, 4, 4))

    elif config == "llll":
        gk = cov_metric_tensor(r=r, theta=theta, a=a, M=M)
        return np.einsum("ij,jklm->iklm", gk, rijkl) + np.zeros((4, 4, 4, 4))

    # elif config=="lluu":
    #     #!Config: lluu
    #     gkinv = cot_metric_tensor(r=r, theta=theta, a=a, M=M)
    #     gk = cov_metric_tensor(r=r, theta=theta, a=a, M=M)
    #     rllll = np.einsum("ij,jklm->iklm", gk, rijkl)

    #     return np.einsum("kjlm,la,mb->kjab", rllll, gkinv, gkinv)

    else:
        return "Invalid String"

In [10]:
def spintensor(levi, pvector, svector, m):

    # p_vectorl = np.einsum("ij,j->i", gk, pvector)  #! Lower the pvector
    # s_vectorl = np.einsum("ij,j->i", gk, svector)  #! Lower the svector

    #! Using above definition
    return -np.einsum("uvab,a,b->uv", levi, svector, pvector) / m

In [11]:
def metric_D(r, theta, a, M=1.0):
    dgkt = np.zeros((4, 4), dtype=type(r))
    dgkp = np.zeros((4, 4), dtype=type(r))

    denom = (r**2 + a**2 * sme.cos(theta) ** 2) ** 2

    dgkt[1, 0] = M * (a**2 - 2 * r**2 + a**2 * sme.cos(2 * theta)) / denom
    dgkt[1, 3] = (
        -(2 * a * M * (-(r**2) + a**2 * sme.cos(theta) ** 2) * sme.sin(theta) ** 2)
        / denom
    )
    dgkt[2, 0] = 2 * a**2 * M * r * sme.sin(2 * theta) / denom
    dgkt[2, 3] = -2 * a * M * r * (r**2 + a**2) * sme.sin(2 * theta) / denom

    dgkp[1, 0] = (
        -2 * a * M * (-(r**2) + a**2 * sme.cos(theta) ** 2) * sme.sin(theta) ** 2 / denom
    )

    dgkp[1, 3] = (
        2
        * sme.sin(theta) ** 2
        * (
            sme.cos(theta) ** 2
            * (2 * a**2 * r * (a**2 + r**2) + a**4 * (M - r) * sme.sin(theta) ** 2)
            + r * (-(a**4) + r**4 + a**2 * (a**2 - M * r) * sme.sin(theta) ** 2)
        )
        / denom
    )

    dgkp[2, 0] = -2 * a * M * r * (a**2 + r**2) * sme.sin(2 * theta) / denom
    dgkp[2, 3] = (
        (
            3 * a**6
            + 10 * a**4 * M * r
            + 11 * a**4 * r**2
            + 16 * a**2 * M * r**3
            + 16 * a**2 * r**4
            + 8 * r**6
            + 4
            * a**2
            * (a**2 + 2 * r**2)
            * (a**2 + r * (-2 * M + r))
            * sme.cos(2 * theta)
            + a**4 * (a**2 - 2 * M * r + r**2) * sme.cos(4 * theta)
        )
        * sme.sin(2 * theta)
        / (8 * denom)
    )

    return dgkt.T + np.zeros((4, 4)), dgkp.T + np.zeros((4, 4))

In [12]:
# @snoop
def energy(pvector, Stensor, dgkt):
    #! Changed Here
    # pvecl = np.einsum("ij,j->i", gk, pvector)
    term2 = np.einsum("uv,uv->", dgkt, Stensor) / 2.0
    return -pvector[0] + term2

In [13]:
def angular_momentum(pvector, Stensor, dgkp):
    #! Changed Here
    # pvecl = np.einsum("ij,j->i", gk, pvector)
    term2 = np.einsum("uv,uv->", dgkp, Stensor) / 2.0
    return pvector[3] - term2

In [14]:
def spin_matrix(sa, pb, epsilon, m):
    """
    Resultant Config: aa
    Input Config: u,u,uu,uuuu
    In Tetrad Basis or Coordinate basis
    Output Config: uu
    Gives same output as above
    """
    #! Changed Sign
    #! Removed Metric Tensor
    # gkinv = sp.linalg.inv(gk)
    #! Changed here
    # return np.einsum("ab,cd,bdkl,k,l->ac",gkinv,gkinv,epsilon,sa,pb)/m
    return -np.einsum("bdkl,k,l->bd", epsilon ,sa ,pb) / m

In [15]:
def to_cartesian(rsol):
    r, t, p = rsol[:, 1], rsol[:, 2], rsol[:, 3]

    x = r * np.sin(t) * np.cos(p)
    y = r * np.sin(t) * np.sin(p)
    z = r * np.cos(t)

    return np.concatenate((x[:, None], y[:, None], z[:, None]), axis=1)

In [16]:
# @snoop
def calculate_four_velocity(gk,gkinv,pvector,svector, ddRuuuu):
    # scalar_divisor = np.einsum("uvps,uv,ps->", Riemann_cov, Stensor, Stensor) / 4.0


    # correction = np.einsum(
    #     "ab,bguv,g,uv->a", Stensor, Riemann_cov, pvector, Stensor
    # ) / (2.0 * (m**2 + scalar_divisor))
    correction= -np.einsum("uabg,a,b,g->u",ddRuuuu,svector,pvector,svector)
    pvecu= np.einsum("ij,j->i",gkinv,pvector)

    dx = (pvecu + correction)
    Vsq = np.einsum("uv,u,v->",gk,dx,dx)

    Pv= -sme.sqrt(-1/Vsq)
    # Pv = 1.0
    dx = dx * Pv

    return dx

In [17]:
def calculate_four_spin(pvector, uvector, svector, cs,dRuluu):

    # correction= -np.einsum("u,abps,psgd,a,b,g,d->u",pvector,Riemann,svector,uvector,pvector,svector,optimize=True)/2.0
    correction= -np.einsum("u,abgd,a,b,g,d->u",pvector,dRuluu,svector,uvector,pvector,svector,optimize=True)

    ds = correction + np.einsum("auv,a,u->v", cs, svector, uvector)

    return ds

In [18]:
def calculate_four_momentum(pvector, uvector, svector, cs, dRlluu):
    # correction = np.einsum(
    #     "abps,psuv,b,u,v->a", Riemann, levi_mixed, uvector, svector, pvector
    # ) / (2.0 * m)
    correction= -np.einsum("uvab,v,a,b->u",dRlluu,uvector,pvector,svector,optimize=True)
    dp = correction + np.einsum("auv,a,u->v", cs, pvector, uvector)

    return dp

In [19]:
def MPD(t, y, p):
    t, r, theta, phi, pt, pr, ptheta, pphi, st, sr, stheta, sphi = y

    a, m, M = p

    # xvector=y[:4]
    pvector = y[4:8]  #! p_u
    svector = y[8:]  #! s_u

    gk = cov_metric_tensor(r, theta, a, M)  #! g_{ij}
    gkinv = cot_metric_tensor(r, theta, a, M)
    cs = kerr_christoffel(r, theta, a, M)  #! G^i_{jk}

    Riemann = kerr_riemann_tensor(r, theta, a, M, "ulll")  #! R^a_{bcd}

    # Riemann_cov = np.einsum("ul,lvps->uvps", gk, Riemann)  #! R_{abcd}
    # Riemann_uull = np.einsum("ap,upgd->uagd", gkinv, Riemann)  #! R^{ua}_{gd}

    levi = levi_civita_tensor(gk)  #! e^{abcd}
    # levi_mixed = np.einsum("psxy,xu,yv->psuv", levi, gk, gk)  #! e^{ps}_{uv}
    levi_mixed= np.einsum("px,sy,xyuv->psuv",gk,gk,levi)

    dRuluu= 0.5*np.einsum("abps,psuv->abuv",Riemann,levi)
    dRlluu= np.einsum("ia,abuv->ibuv",gk,dRuluu)

    ddRuuuu= 0.5*np.einsum("bi,aips,psuv->abuv",gkinv,dRuluu,levi_mixed)

    # stensor = spintensor(levi, pvector=pvector, svector=svector, gk=gk, m=m)  #! S^{uv}

    uvector = calculate_four_velocity(gk=gk,gkinv=gkinv,
        pvector=pvector, svector=svector,ddRuuuu=ddRuuuu)  #!dx_mu

    dp = calculate_four_momentum(
        pvector=pvector,
        uvector=uvector,
        svector=svector,
        cs=cs,
        dRlluu=dRlluu
    )
    #!dp_{mu}

    ds = calculate_four_spin(
        pvector=pvector,
        uvector=uvector,
        svector=svector,
        cs=cs,
        dRuluu=dRuluu
    )
    #!ds^{mu}

    dy = np.zeros_like(y)

    dy[0:4] = uvector
    dy[4:8] = dp
    dy[8:] = ds

    return dy

In [20]:
# @snoop
def get_constants(rsol, gk,gkinv, a,m, M=1.0):
    _, r, theta, _ = rsol[:4]
    pmul = rsol[4:8]
    smul = rsol[8:]

    dgkt, dgkp = metric_D(r, theta=theta, a=a, M=M)

    levi = levi_civita_tensor(gk=gk)  #!'uuuu'
    # Sab=spin_matrix(sa=si,pb=pi,gk=eta,epsilon=epsilon)
    Suv = spintensor(levi, pmul, smul, m)

    E = energy(pmul, Suv, dgkt)
    Jz = angular_momentum(pmul, Suv, dgkp)

    mu = get_norm(T=pmul, g=gkinv)
    Smag = get_norm(T=smul, g=gkinv)

    ps = np.einsum("ij,j,i->", gkinv, pmul, smul)

    return E, Jz, mu, Smag, ps

In [21]:
t, r, theta, phi = smp.symbols("t r theta phi")
pt, pr, ptheta, pphi = sme.symbols("pt pr ptheta pphi")
st, sr, stheta, sphi = sme.symbols("st sr stheta sphi")
a, m, M = sme.symbols("a m M")

In [22]:
p=[a,m,M]

In [23]:
y0=np.array([t,r,theta,phi,pt,pr,ptheta,pphi,st,sr,stheta,sphi])

In [28]:
f=MPD(0.,y0,p)

In [29]:
Vy0 = [
    0.0,
    6.0,
    1.5707963267948966,
    0.0,
    -0.930946947662005,
    0.01,
    -1.374960485380533,
    2.651574532312764,
    0.39585430222598117,
    1e-06,
    1e-06,
    -6.597950372340081,
]

In [30]:
Vp=[0.8,1.0,1.0]

In [31]:
Vf = [
    f[i].subs(
        {
            t: Vy0[0],
            r: Vy0[1],
            theta: Vy0[2],
            phi: Vy0[3],
            pt: Vy0[4],
            pr: Vy0[5],
            ptheta: Vy0[6],
            pphi: Vy0[7],
            st: Vy0[8],
            sr: Vy0[9],
            stheta: Vy0[10],
            sphi: Vy0[11],
            a: Vp[0],
            M: Vp[2],
        }
    )
    for i in range(len(f))
]

In [32]:
Vf

[-1.36479838292298,
 -0.00684081462861727,
 0.0388088270750185,
 -0.0818831984320458,
 -2.21514381913117e-05,
 0.00211716525838047,
 -9.43717016094741e-05,
 -2.72485030288557e-05,
 -8.72011677701141e-05,
 0.076296320877887,
 -2.38329558163825e-05,
 0.00760593654722135]

In [20]:
M = 1e0
m = 1e0

S = 1e0*m*M #1e0 * m * M
E = 0.9328 * m
Jz = 2.8 * m * M

r0 = 6.0 * M
theta0 = np.pi / 2.0
phi0 = 0.0
a0 = 0.8 * M

In [21]:
v_ = np.sqrt(M / r0)
abar = a0 / M

Epro = (1 - 2 * v_**2 + abar * v_**3) / (np.sqrt(1 - 3 * v_**2 + 2 * abar * v_**3))
Eret = (1 - 2 * v_**2 - abar * v_**3) / (np.sqrt(1 - 3 * v_**2 - 2 * abar * v_**3))

ideg = np.pi / 30
ecc = 0.01
alpha_ip = 1 + np.cos(ideg)
alpha_im = 1 - np.cos(ideg)

E = 0.5 * (alpha_ip * Epro + alpha_im * Eret)
Jz = np.cos(ideg) * np.sqrt((1 - ecc**2) / (2 * (1 - E)))

In [22]:
E,Jz

(0.9242829878465502, 2.5555295156967808)

In [23]:
gk = cov_metric_tensor(r=r0, theta=theta0, a=a0, M=M)
gkinv = cot_metric_tensor(r=r0, theta=theta0, a=a0, M=M)

In [24]:
cs = kerr_christoffel(r=r0, theta=theta0, a=a0, M=M)

In [25]:
# rllll = kerr_riemann_tensor(r=r0, theta=theta0, a=a0, M=M, config="llll")
rulll = kerr_riemann_tensor(r=r0, theta=theta0, a=a0, M=M, config="ulll")
# ruull=np.einsum("ij,ujps->uips",gkinv,rulll)

In [26]:
levi = levi_civita_tensor(gk)
#!epsilon^{uuuu}

In [27]:
levi_mixed = np.einsum("px,sy,xyuv->psuv", gk, gk, levi)

In [28]:
dgkt, dgkp = metric_D(r0, theta0, a0, M)

In [29]:
P1 = 1e-2 * m

In [30]:
S1 = 1e-6 * m
S2 = 1e-6 * m

In [31]:
p0, p2, p3 = sme.symbols("p0 p2 p3", real=True)
#!Coordinate Components, 'l'

In [32]:
s0, s3 = sme.symbols("s0 s3", real=True)
#!Coordinate Components, 'l'

In [33]:
gpcoordu = np.array([p0, P1, p2, p3])
gscoordu = np.array([s0, S1, S2, s3])

#!Config='l'

In [34]:
# Definitions
Delta = a0**2 - 2 * M * r0 + r0**2
Sigma = r0**2 + a0**2 * sme.cos(theta0) ** 2

In [35]:
# Sab = spi(sa=gsi, pb=gpi, gk=eta, epsilon=epsl)
Suv = spintensor(levi=levi, pvector=gpcoordu, svector=gscoordu, m=m)
#!Config: uu, In Coordinate Basis

In [36]:
smp.Matrix(Suv).expand()

Matrix([
[                                                 0,    0.0277777777777778*p2*s3 - 2.77777777777778e-8*p3,    2.77777777777778e-8*p3 - 0.000277777777777778*s3,      2.77777777777778e-10 - 2.77777777777778e-8*p2],
[-0.0277777777777778*p2*s3 + 2.77777777777778e-8*p3,                                                    0, 0.0277777777777778*p0*s3 - 0.0277777777777778*p3*s0, -2.77777777777778e-8*p0 + 0.0277777777777778*p2*s0],
[ -2.77777777777778e-8*p3 + 0.000277777777777778*s3, -0.0277777777777778*p0*s3 + 0.0277777777777778*p3*s0,                                                   0,   2.77777777777778e-8*p0 - 0.000277777777777778*s0],
[     2.77777777777778e-8*p2 - 2.77777777777778e-10,    2.77777777777778e-8*p0 - 0.0277777777777778*p2*s0,   -2.77777777777778e-8*p0 + 0.000277777777777778*s0,                                                  0]])

In [37]:
eq210a = energy(gpcoordu, Suv, dgkt) - E

In [38]:
eq210a.expand()

-0.92428298784655 - 0.999999999382716*p0 + 7.71604938271605e-10*p3 - 4.61636406592171e-21*s0 - 1.00793975238465e-22*s3 - 0.000617283950617284*p2*s0 - 0.000771604938271605*p2*s3

In [39]:
eq210b = angular_momentum( gpcoordu, Suv, dgkp) - Jz

In [40]:
eq210b.expand()

-2.55552951569678 - 1.66172839506173e-07*p0 + 1.00000000061728*p3 - 6.30530823260753e-19*s0 - 4.61636406592171e-21*s3 + 0.166172839506173*p2*s0 - 0.000617283950617284*p2*s3

In [41]:
gpcoordu

array([p0, 0.01, p2, p3], dtype=object)

In [42]:
eq533 = get_norm(T=gpcoordu, g=gkinv) + m**2

In [43]:
eq229 = np.einsum("ij,j,i->", gkinv, gpcoordu, gscoordu) - 0.0

In [44]:
eq228 = np.einsum("ij,j,i->", gkinv, gscoordu, gscoordu) - S**2

In [45]:
sme.sympify(eq533)

1.00006844444444 - 0.0216450216450216*p0*p3 - 1.495670995671*p0**2 + 0.0277777777777778*p2**2 + 0.0270562770562771*p3**2

In [46]:
eq210a.expand()

-0.92428298784655 - 0.999999999382716*p0 + 7.71604938271605e-10*p3 - 4.61636406592171e-21*s0 - 1.00793975238465e-22*s3 - 0.000617283950617284*p2*s0 - 0.000771604938271605*p2*s3

In [47]:
eq228.expand()

-0.999999999999288 - 0.0216450216450216*s0*s3 - 1.495670995671*s0**2 + 0.0270562770562771*s3**2

In [48]:
eq229.expand()

6.84444444444444e-09 + 2.77777777777778e-08*p2 - 1.495670995671*p0*s0 - 0.0108225108225108*p0*s3 - 0.0108225108225108*p3*s0 + 0.0270562770562771*p3*s3

In [49]:
eq210b.expand()

-2.55552951569678 - 1.66172839506173e-07*p0 + 1.00000000061728*p3 - 6.30530823260753e-19*s0 - 4.61636406592171e-21*s3 + 0.166172839506173*p2*s0 - 0.000617283950617284*p2*s3

In [50]:
# @snoop(depth=1,watch='x')
def solve_eq(p_0, p_2, p_3, s_0, s_3):
    f = smp.lambdify([p0, p2, p3, s0, s3], [eq210a, eq210b, eq533, eq228, eq229])
    return f(p_0, p_2, p_3, s_0, s_3)


# f = smp.lambdify([p0, p2, p3, s0, s3], [eq210a, eq210b, eq533, eq228, eq229])

In [51]:
# @snoop(depth=1,watch='x')
def solve_eq_sp(p):
    f = sme.lambdify([p0, p2, p3, s0, s3], [eq210a, eq210b, eq533, eq228, eq229])
    return np.array(f(p[0], p[1], p[2], p[3], p[4]))


# f = smp.lambdify([p0, p2, p3, s0, s3], [eq210a, eq210b, eq533, eq228, eq229])

In [52]:
def jac_nl_symb():
    X = smp.Matrix([eq210a, eq210b, eq533, eq228, eq229])
    Y = smp.Matrix([p0, p2, p3, s0, s3])

    return X.jacobian(Y)

In [53]:
jac = jac_nl_symb()

In [54]:
def jac_nl(p_0, p_2, p_3, s_0, s_3):
    f = smp.lambdify([p0, p2, p3, s0, s3], jac)

    return f(p_0, p_2, p_3, s_0, s_3)

In [55]:
def jac_nl_sp(p):
    f = sme.lambdify([p0, p2, p3, s0, s3], jac)

    return f(p[0], p[1], p[2], p[3], p[4])

In [56]:
solve_eq(-0.924284, -1.33974, 2.55554, 0.0000381344, -0.000652199)

[3.708812040814635e-07,
 1.6103025708424923e-06,
 -9.986994939925609e-07,
 -0.9999999901272496,
 1.3622010642796498e-08]

In [57]:
roots = mpmath.findroot(
    solve_eq,
    (-0.92,-1.33,2.55,0.0000038,-0.00065),
    solver="mdnewton",
    tol=1e-19,
    J=jac_nl,
    verify=True,
)

In [58]:
roots = np.array(roots.tolist(), dtype=np.float64)[:, 0]

In [59]:
solve_eq(*roots)

[0.0,
 0.0,
 -4.440892098500626e-16,
 1.1102230246251565e-16,
 4.4310786593238686e-17]

In [60]:
roots

array([-0.93094695, -1.37496049,  2.65157453,  0.3958543 , -6.59795037])

In [61]:
np0, np2, np3, ns0, ns3 = roots

In [62]:
ngpcoordu = np.array([np0, P1, np2, np3]).astype(np.float64)
ngscoordu = np.array([ns0, S1, S2, ns3]).astype(np.float64)

In [63]:
ngpcoordu, ngscoordu

(array([-0.93094695,  0.01      , -1.37496049,  2.65157453]),
 array([ 3.95854302e-01,  1.00000000e-06,  1.00000000e-06, -6.59795037e+00]))

In [64]:
Suv = spintensor(levi, ngpcoordu, ngscoordu, m)

In [65]:
Suv

array([[-0.00000000e+00,  2.51997733e-01,  1.83283765e-03,
         3.84711246e-08],
       [-2.51997733e-01, -0.00000000e+00,  1.41464016e-01,
        -1.51189748e-02],
       [-1.83283765e-03, -1.41464016e-01, -0.00000000e+00,
        -1.09985388e-04],
       [-3.84711246e-08,  1.51189748e-02,  1.09985388e-04,
        -0.00000000e+00]])

In [66]:
get_norm(ngpcoordu, gkinv)

-0.9999999999999925

In [67]:
get_norm(ngscoordu, gkinv)

0.9999999999999992

In [68]:
dRuluu= 0.5*np.einsum("abps,psuv->abuv",rulll,levi)
ddRuuuu= 0.5*np.einsum("bi,aips,psuv->abuv",gkinv,dRuluu,levi_mixed)

In [69]:
dRlluu = np.einsum("ia,abuv->ibuv", gk, dRuluu)

In [70]:
vmu0= calculate_four_velocity(gk=gk,gkinv=gkinv,pvector=ngpcoordu,svector=ngscoordu,ddRuuuu=ddRuuuu)

In [71]:
vmu0

array([-1.36479838, -0.00684081,  0.03880883, -0.0818832 ])

In [72]:
calculate_four_momentum(ngpcoordu,vmu0,ngscoordu,cs,dRlluu)

array([-2.21514382e-05,  2.11716526e-03, -9.43717016e-05, -2.72485030e-05])

In [73]:
calculate_four_spin(ngpcoordu,vmu0,ngscoordu,cs,dRuluu)

array([-8.72011678e-05,  7.62963209e-02, -2.38329558e-05,  7.60593655e-03])

In [74]:
#!The initial conditions
#!We pass r0^mu,p^mu and S^mu
y0 = np.array(
    [
        0.0,
        r0,
        theta0,
        phi0,
        ngpcoordu[0],
        ngpcoordu[1],
        ngpcoordu[2],
        ngpcoordu[3],
        ngscoordu[0],
        ngscoordu[1],
        ngscoordu[2],
        ngscoordu[3],
    ]
).astype(np.float64)

In [79]:
y0.tolist()

[0.0,
 6.0,
 1.5707963267948966,
 0.0,
 -0.930946947662005,
 0.01,
 -1.374960485380533,
 2.651574532312764,
 0.39585430222598117,
 1e-06,
 1e-06,
 -6.597950372340081]

In [76]:
p = [a0, m, M]

In [77]:
p

[0.8, 1.0, 1.0]

In [78]:
MPD(0.0, y0, p)

array([-1.36479838e+00, -6.84081463e-03,  3.88088271e-02, -8.18831984e-02,
       -2.21514382e-05,  2.11716526e-03, -9.43717016e-05, -2.72485030e-05,
       -8.72011678e-05,  7.62963209e-02, -2.38329558e-05,  7.60593655e-03])

In [ ]:
get_constants(y0, gk,gkinv, a0, m, M)

In [ ]:
def poincare(t, y, *args):
    # return y[3]%np.pi -1.
    if (y[2] - (np.pi / 2)) > 0:
        return 1
        # return y[1]
        # return rhs27(y[:4],y[4:8]/m,y[8:],metric_tensor(y[1],y[2],a0,M),epsl,a0,m,"u",M)[2]-np.pi/2
    else:
        return -1  # Impossible Conditions

In [ ]:
np.arange(0.,100.0,1e-6).shape

In [ ]:
# poincare.direction =0

In [ ]:
sol = solve_ivp(
    fun=MPD,
    t_span=[0.0, 50000.0],
    y0=y0,
    method="LSODA",
    t_eval=np.arange(0.0, 50000.0 + 1, 1),
    events=poincare,
    args=(p,),
    rtol=1e-12,
    atol=1e-12,
)

In [ ]:
rsol = sol.y.T

In [ ]:
i = -1
gki = cov_metric_tensor(rsol[i][1], rsol[i][2], a0, M)
gkinvi = cot_metric_tensor(rsol[i][1], rsol[i][2], a0, M)
(
    get_constants(rsol=rsol[i], gk=gki,gkinv=gkinvi, a=a0, m=m, M=M)[0] - E,
    get_constants(rsol=rsol[i], gk=gki,gkinv=gkinvi, a=a0, m=m, M=M)[1] - Jz,
    get_constants(rsol=rsol[i], gk=gki,gkinv=gkinvi, a=a0, m=m, M=M)[2] + m**2,
    get_constants(rsol=rsol[i], gk=gki,gkinv=gkinvi, a=a0, m=m, M=M)[3] - S**2,
)

In [ ]:
motconst = np.array(
    [
        get_constants(
            rsol[i],
            cov_metric_tensor(rsol[i][1], rsol[i][2], a0, M),
            cot_metric_tensor(rsol[i][1], rsol[i][2], a0, M),
            a0,
            m,
            M,
        )
        - np.array([E, Jz, -(m**2), S**2, 0.0])
        for i in range(len(rsol))
    ]
)

In [ ]:
f, ax = plt.subplots(3, 2, figsize=(12, 15))
ax[0, 0].plot(sol.t, motconst[:, 0], label="Energy", color="blue")
ax[0, 1].plot(sol.t, motconst[:, 1], label="Jz", color="green")
ax[1, 0].plot(sol.t, motconst[:, 2], label="m^2", color="red")
ax[1, 1].plot(sol.t, motconst[:, 3], label="S^2", color="purple")
ax[2, 0].plot(sol.t, motconst[:, 4], label=r"$p_v S^v$", color="orange")
ax[0, 0].legend(loc="upper right")
ax[0, 1].legend(loc="upper right")
ax[1, 0].legend(loc="upper right")
ax[1, 1].legend(loc="upper right")
ax[2, 0].legend(loc="upper right")
f.delaxes(ax[2, 1])

In [ ]:
crsol = to_cartesian(rsol=rsol)

In [ ]:
crsol

In [ ]:
# fig = go.Figure(
#     data=go.Scatter3d(
#         x=crsol[::2, 0],
#         y=crsol[::2, 1],
#         z=crsol[::2, 2],
#         marker=dict(
#             size=4,
#             color=crsol[::2, 2],
#             colorscale="Viridis",
#         ),
#         line=dict(color="darkblue", width=2),
#     )
# )

# fig.update_layout(
#     title=f"Particle motion with the initial conditions E={E:.3E}, Jz={Jz:.3E} and S={S:.3E}",
#     width=800,
#     height=700,
#     autosize=False,
#     scene=dict(
#         camera=dict(
#             up=dict(x=0, y=0, z=1),
#             eye=dict(
#                 x=0,
#                 y=1.0707,
#                 z=1,
#             ),
#         ),
#         aspectratio=dict(x=1, y=1, z=0.7),
#         aspectmode="manual",
#     ),
# )

# # fig.update_traces(line_colorbar_exponentformat="E", selector=dict(type='parcoords'))

# fig.show()

In [ ]:
poi_points = sol.y_events[0]

In [ ]:
poi_points.shape

In [ ]:
vr = np.zeros(poi_points[:, 5].shape[0])

In [ ]:
vr = np.zeros(poi_points[:, 5].shape[0])
for i in range(len(vr)):
    gkp = cov_metric_tensor(poi_points[i, 1], poi_points[i, 2], a0, M)
    gkinvp = cot_metric_tensor(poi_points[i, 1], poi_points[i, 2], a0, M)
    levip = levi_civita_tensor(gkp)
    levi_mixedp = np.einsum("px,sy,xyuv->psuv", gkp, gkp, levip)
    # levi_mixedp = np.einsum("psxy,xu,yv->psuv", levip, gkp, gkp)
    # Stensorp = spintensor(levip, poi_points[i][4:8], poi_points[i][8:], gkp, m)
    rulllp = kerr_riemann_tensor(poi_points[i][1], poi_points[i][2], a0, M, "ulll")
    dRuluup = 0.5 * np.einsum("abps,psuv->abuv", rulllp, levip)
    ddRuuuup = 0.5 * np.einsum("bi,aips,psuv->abuv", gkinvp, dRuluup, levi_mixedp)
    vr[i] = calculate_four_velocity(
        gk=gkp,
        gkinv=gkinvp,
        pvector=poi_points[i][4:8],
        svector=poi_points[i][8:],
        ddRuuuu=ddRuuuup,
    )[1]

In [ ]:
plt.figure(dpi=1200)
plt.scatter(poi_points[:, 1], vr, s=0.5, c="black")
plt.title(r"Poincare plot in $\theta = \pi/2$ plane")
plt.xlabel("r")
plt.ylabel(r"$V_r$")
# plt.savefig("Poincareplot_python.png")
# plt.xlim(9,10)
# plt.ylim(-4,-3)

In [ ]:
def Correlation_dimension(data, r=0.01):
    new_t = KDTree(data)

    sum = len(new_t.query_pairs(r, output_type="ndarray"))
    Corr_dimension = sum / len(data) ** 2

    return Corr_dimension

In [ ]:
r = np.arange(0.0001, 1, 0.01)

In [ ]:
cd001 = []
for i in range(len(r)):
    cd001.append(Correlation_dimension(crsol, r[i]))

In [ ]:
plt.plot(np.log10(r), np.log10(cd001))

In [ ]:
intcept, Codim = (
    np.polynomial.polynomial.Polynomial.fit(np.log10(r[8:]), np.log10(cd001[8:]), 1)
    .convert()
    .coef
)

In [ ]:
Codim

## Create a Deviation Vector satisfying Constrains

In [ ]:
dp0, dp3, ds0 = smp.symbols("dp0 dp3 ds0")
# ds0=0.0

In [ ]:
EPS=1e-13

In [ ]:
rany0 = (
    np.random.uniform(
        0.0,
        1.0,
        (12,),
    ).astype(object)
    * EPS
)
rany0[[0, 4, 7, 8]] = [0.0, dp0, dp3, ds0]

In [ ]:
rany0

In [ ]:
ry0 = y0 + rany0

In [ ]:
rgkinv= cot_metric_tensor(ry0[1],ry0[2],a0,M)

In [ ]:
eqC1 = get_norm(ry0[4:8], rgkinv) + m**2

In [ ]:
eqC1

In [ ]:
eqC2 = get_norm(ry0[8:], rgkinv) - S**2

In [ ]:
eqC3 = np.einsum("ij,j,i->", rgkinv, ry0[4:8], ry0[8:])

In [ ]:
eqC3

In [ ]:
eqC2.simplify()

In [ ]:
Croots = smp.solve([eqC1, eqC2, eqC3], [dp0, dp3, ds0])
# Croots = smp.solve([eqC1, eqC3], [dp0, dp3])
display(Croots)

In [ ]:
ndp0, ndp3, nds0 = Croots[0]

In [ ]:
rany0[[4, 7, 8]] = [ndp0, ndp3, nds0]

In [ ]:
ry0 = (y0 + rany0).astype(float)

In [ ]:
ry0

In [ ]:
dy0=ry0-y0

In [ ]:
dy0

In [ ]:
np.linalg.norm(dy0)

### Using Approximate Jacobian

In [ ]:
EPS = np.finfo(float).eps
H = np.diag(np.ones_like(y0) * EPS * 1j)

In [ ]:
def fjac(t, x, p):
    x = x.astype(np.complex128)
    # fx = mpd(0, x, p)
    n = 12
    J = np.zeros((n, n))

    for i in range(n):
        J[:, i] = MPD(0, x + H[i], p).imag / H[i, i].imag

    return J.T

In [ ]:
def solve_combined(t, y, p):
    mpdarr = MPD(t, y[:12], p)
    jac = fjac(t, y[:12], p)

    dY = y[12:]

    dY_dt = np.einsum("ij,j->i", jac, dY)

    dydtf = dY_dt.flatten()

    du_ = np.zeros((24,))

    du_[:12] = mpdarr
    du_[12:] = dydtf

    return du_

In [ ]:
w0 = dy0

Delta = a0**2 - 2 * M * r0 + r0**2
Sigma = r0**2 + a0**2 * np.cos(theta0) ** 2
scA= (r0**2+a0**2)**2-Delta*a0**2*np.sin(theta0)**2
Wk= 2*M*a0*r0/scA

Umu= np.array([1,0,0,Wk])/np.sqrt(Delta*Sigma/scA)

Puv= np.eye(4)+np.einsum("u,v->uv",Umu,Umu)

xi= np.einsum("iu,u->i",Puv,y0[:4])[1:]
pi= np.einsum("ui,i->u",Puv,y0[4:8])[1:]
si = np.einsum("ui,i->u", Puv, y0[8:])[1:]

alphak= np.linalg.norm(np.array([xi,pi,si]).flatten())
# w0 = w0 / np.linalg.norm(w0)
w0= w0/alphak

In [ ]:
w0

In [ ]:
ics0 = np.array((y0, w0)).flatten()

In [ ]:
(solve_combined(0.0, ics0, p))

In [ ]:
tau = 100 * M  # Renormalization time
tf = 10000.0  # Final time
steps = 100

In [ ]:
# @nb.njit
# @snoop
def evolve_lyap(ics0, tau, tf, steps, p):

    ti = 0.0  # * Initial Time
    timestep = np.arange(ti, tf + tau, tau)  # * Time interval
    size = timestep.shape[0]

    X1t = np.zeros((size - 1))  # *Lyapunaov Exponents
    r1t = np.zeros((size - 1))
    sum = 0.0
    ics = ics0.copy()  # *We do not want to change our initial conditions
    # h = np.zeros(size - 1)  # *Check for energy
    # sol = np.zeros((size, 12))
    # sol[0, :] = ics[:12]
    # traj=np.zeros((steps,6),dtype=np.float64)

    for i in range(0, size-1): #size-1
        timeint = np.linspace(
            timestep[i], timestep[i + 1], steps
        )  #!Our time interval with 10000 points between them

        # Uncomment following line to use scipy solver and comment above two lines
        usol = solve_ivp(
            solve_combined,
            [timeint[0], timeint[-1]],
            ics,
            "LSODA",
            t_eval=timeint,
            args=(p,),
            atol=1e-12,
            rtol=1e-12,
        ).y[:, -1]

        xk = usol[:12]  # Orbit at t= tau
        wk = usol[12:]  # Deviations at t=tau

        # *---------------------The following algorithm is from the paper

        # alphak = np.linalg.norm(wk)
        #! Change Norm
        r,theta= xk[1],xk[2]

        Delta = a0**2 - 2 * M * r + r**2
        Sigma = r**2 + a0**2 * np.cos(theta) ** 2
        scA= (r**2+a0**2)**2-Delta*a0**2*np.sin(theta)**2
        Wk= 2*M*a0*r/scA

        Umu= np.array([1,0,0,Wk])/np.sqrt(Delta*Sigma/scA)

        Puv= np.eye(4)+np.einsum("u,v->uv",Umu,Umu)

        xi= np.einsum("iu,u->i",Puv,xk[:4])[1:]
        pi= np.einsum("ui,i->u",Puv,xk[4:8])[1:]
        si = np.einsum("ui,i->u", Puv, xk[8:])[1:]

        alphak= np.linalg.norm(np.array([xi,pi,si]).flatten())

        sum = sum + np.log(alphak)
        # X1t[i] = sum / timestep[i + 1]
        r1t[i]= sum
        wk0 = wk / alphak

        # *----------------------Lyapunaov Exponents calculated above
        #! Now for another time step. Set the current solution as the initial conditions for next time steps
        ics[:12] = xk
        ics[12:] = wk0

        #! ------------------------Sanity Check--------------------------
        # if sanity_check!=0:
        # h[i] = H(1.00, ics, alpha)  #! Current Energy
        # sol[i + 1, :] = ics[:12]
        #! Comment above line if you do not want to perform the sanity check

    # if sanity_check!=0:
    #     return X1t,h,sol, traj

    return r1t #X1t  # , h, sol

In [ ]:
ritau = evolve_lyap(ics0, tau, tf, steps, p)

In [ ]:
ritau

In [ ]:
plt.plot(np.arange(0.0, tf, tau), ritau)

In [ ]:
intcept, Lyaps = (
    np.polynomial.polynomial.Polynomial.fit(
        np.arange(0.0, tf, tau), ritau, 1
    )
    .convert()
    .coef
)

In [ ]:
Lyaps

# Constraint Deviation Vectors

In [ ]:
dp0, dp3, ds0 = smp.symbols("dp0 dp3 ds0")
# ds0=0.0

In [ ]:
EPS = 1e-7

In [ ]:
rany0 = (
    np.random.uniform(
        0.0,
        1.0,
        (12,),
    ).astype(object)
    * EPS
)
rany0[[0, 4, 7, 8]] = [0.0, dp0, dp3, ds0]

In [ ]:
rany0

In [ ]:
ry0 = y0 + rany0

In [ ]:
ry0

In [ ]:
rgkinv = cot_metric_tensor(ry0[1], ry0[2], a0, M)

In [ ]:
eqC1 = get_norm(ry0[4:8], rgkinv) + m**2

In [ ]:
eqC2 = get_norm(ry0[8:], rgkinv) - S**2

In [ ]:
str(eqC2.simplify())

In [ ]:
eqC3 = np.einsum("ij,j,i->", rgkinv, ry0[4:8], ry0[8:])

In [ ]:
str(eqC3.simplify())

In [ ]:
const_roots = smp.solve([eqC1, eqC2, eqC3], [dp0, dp3, ds0])

In [ ]:
const_roots

In [ ]:
eqC1.subs(
    [(dp0, const_roots[0][0]), (dp3, const_roots[0][1]), (ds0, const_roots[0][2])]
), \
eqC2.subs(
    [(dp0, const_roots[0][0]), (dp3, const_roots[0][1]), (ds0, const_roots[0][2])]
), \
eqC3.subs(
    [(dp0, const_roots[0][0]), (dp3, const_roots[0][1]), (ds0, const_roots[0][2])]
)

In [ ]:
y0

In [ ]:
ndp0, ndp3, nds0 = const_roots[0]

In [ ]:
# ry0[[4,6,8]]=dp0,dp2,ds0
rany0[[4, 7, 8]] = ndp0, ndp3, nds0

In [ ]:
ry0 = (y0 + rany0).astype(float)

In [ ]:
dy0 = ry0 - y0

In [ ]:
inorm= np.linalg.norm(dy0)

In [ ]:
inorm

In [ ]:
cinorm=np.linalg.norm(dy0[[True, True, True, True, False, True, True, False, False, True, True, True]])

In [ ]:
dy0,cinorm

In [ ]:
tau = 100 * M
tf = 20000

In [ ]:
def func_build(delyr,y0): 
    # y0=pam["y"]
    def mp_solve_const(x_0,x_1,x_2):

        delyr[[4, 7, 8]] = [x_0,x_1,x_2]

        ry0 = y0 + delyr
        rgkinv = cot_metric_tensor(ry0[1], ry0[2], a0, M)
        eqC1 = get_norm(ry0[4:8], rgkinv) + m**2
        eqC2 = get_norm(ry0[8:], rgkinv) - S**2
        eqC3 = np.einsum("ij,j,i->", rgkinv, ry0[4:8], ry0[8:])

        return [eqC1,eqC2,eqC3]
    
    return mp_solve_const
    

In [ ]:
# @snoop
def evolve_lyap2(y0, y1, p):
    ti = 0.0
    timestep = np.arange(ti, tf + tau, tau)

    re= np.zeros((len(timestep)))
    mask = np.array(
        [True, True, True, True, False, True, True, False, False, True, True, True]
    )

    cinorm= np.linalg.norm((y1-y0)[mask])

    for i in range(len(timestep)-1):
        t_span= [0.,tau]
        soly0 = solve_ivp(
            fun=MPD,
            t_span=t_span,
            y0=y0,
            method="LSODA",
            t_eval=[tau],
            args=(p,),
            rtol=1e-12,
            atol=1e-12,
        ).y[:,-1]

        soly1 = solve_ivp(
            fun=MPD,
            t_span=t_span,
            y0=y1,
            method="LSODA",
            t_eval=[tau],
            args=(p,),
            rtol=1e-12,
            atol=1e-12,
        ).y[:,-1]

        re[i+1]=np.log(np.linalg.norm((soly1-soly0)[mask])/cinorm)

        delyr = ((soly1 - soly0))/np.linalg.norm((soly0 - soly1)[mask])
        delyr[~mask]=0.0
        delyr=delyr*cinorm
        # print(np.where()[0])
        # delyr=np.insert(delyr,np.where(mask)[0],0.)

        sign= np.sign(soly1[~mask])
        iv= (np.array([EPS,EPS,EPS])*sign).tolist()

        f= func_build(delyr,soly0)
        root_ = mpmath.findroot(
            f, iv, solver="mdnewton", tol=1e-24)
        root = np.array(root_.tolist(), dtype=float)[:, 0]

        delyr[[4,7,8]]= root[0],root[1],root[2]
        y1= soly0+ delyr
        y0=soly0

        dum2_=np.linalg.norm((y1-y0)[mask])

    return re

In [ ]:
lis = evolve_lyap2(y0, ry0, p)

In [ ]:
lis

In [ ]:
# cre= np.cumsum(np.log(lis))

In [ ]:
timestep= np.arange(0,tf+tau,int(tau))

In [ ]:
timestep.shape,lis.shape

In [ ]:
plt.scatter(timestep, np.cumsum(lis), s=0.5)
# plt.scatter(timestep, lis, s=0.5)

In [ ]:
timestep[10:26]

In [ ]:
intcept, lyap = np.polynomial.polynomial.Polynomial.fit(timestep[10:], lis[10:], 1).convert().coef

In [ ]:
lyap

### Get the Derivatives of Dynamical Variables

In [ ]:
def Dcov_metric_tensor(r, theta, a, M=1.0):
    """
    Define the metric tensor function.

    Parameters:
        a (float): Parameter 'a' in the metric.
        r (float): Parameter 'r' in the metric.
        theta (float): Parameter 'theta' in the metric.

    Returns:
        np.ndarray: The metric tensor at the given values of a, r, and theta.
        Configuration: ll
    """
    #! Correction Added Mass term in the symbols
    dg = np.zeros((4, 4, 4), dtype=type(r))

    # Definitions
    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * np.cos(theta) ** 2
    dSt = -(a**2) * np.sin(2 * theta)
    dSr = 2 * r
    dD = 2 * (r - M)

    dg[0, 0, [[1, 2]]] = [
        2.0 * M * (-r * dSr + Sigma) / Sigma**2,
        -2.0 * M * r * dSt / Sigma**2,
    ]

    dg[1, 1, [[1, 2]]] = [(Delta * dSr - Sigma * dD) / Delta**2, dSt / Delta]

    dg[2, 2, [[1, 2]]] = [dSr, dSt]

    dg[3, 3, [[1, 2]]] = [
        (
            (a**2 * Delta * np.sin(theta) ** 2 - (a**2 + r**2) ** 2) * dSr
            + (-(a**2) * dD * np.sin(theta) ** 2 + 4 * r * (a**2 + r**2)) * Sigma
        )
        * np.sin(theta) ** 2
        / Sigma**2,
        (
            2
            * (-2 * a**2 * Delta * np.sin(theta) ** 2 + (a**2 + r**2) ** 2)
            * Sigma
            * np.cos(theta)
            + (a**2 * Delta * np.sin(theta) ** 2 - (a**2 + r**2) ** 2)
            * dSt
            * np.sin(theta)
        )
        * np.sin(theta)
        / Sigma**2,
    ]

    dg[0, 3, [[1, 2]]] = [
        2.0 * M * a * (r * dSr - Sigma) * np.sin(theta) ** 2 / Sigma**2,
        M
        * a
        * r
        * (-2.0 * Sigma * np.sin(2 * theta) - 1.0 * dSt * np.cos(2 * theta) + 1.0 * dSt)
        / Sigma**2,
    ]

    dg[3, 0, [[1, 2]]] = dg[0, 3, [[1, 2]]]

    # return dg.asty, [[1,2]]pe(float)
    return dg

In [ ]:
def Dcot_metric_tensor(r, theta, a, M=1.0):
    """
    Define the metric tensor function.

    Parameters:
        a (float): Parameter 'a' in the metric.
        r (float): Parameter 'r' in the metric.
        theta (float): Parameter 'theta' in the metric.

    Returns:
        np.ndarray: The metric tensor at the given values of a, r, and theta.
        Configuration: uu
    """
    # Definitions
    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * np.cos(theta) ** 2
    dSt = -(a**2) * np.sin(2 * theta)
    dSr = 2 * r
    dD = 2 * (r - M)

    denom = Delta * np.sin(theta) ** 2
    ddmr = dD * np.sin(theta) ** 2
    ddmt = Delta * np.sin(2 * theta)

    dg = np.zeros((4, 4, 4), dtype=type(r))

    dg[0, 0, [[1, 2]]] = [
        (
            -(a**2 * Delta * np.sin(theta) ** 2 - (a**2 + r**2) ** 2) * Sigma * ddmr
            - (a**2 * Delta * np.sin(theta) ** 2 - (a**2 + r**2) ** 2) * dSr * denom
            + (a**2 * dD * np.sin(theta) ** 2 - 4 * r * (a**2 + r**2)) * Sigma * denom
        )
        * np.sin(theta) ** 2
        / (Sigma**2 * denom**2),
        (
            -(a**2 * Delta * np.sin(theta) ** 2 - (a**2 + r**2) ** 2)
            * Sigma
            * ddmt
            * np.sin(theta)
            - (a**2 * Delta * np.sin(theta) ** 2 - (a**2 + r**2) ** 2)
            * dSt
            * denom
            * np.sin(theta)
            + 2
            * (2 * a**2 * Delta * np.sin(theta) ** 2 - (a**2 + r**2) ** 2)
            * Sigma
            * denom
            * np.cos(theta)
        )
        * np.sin(theta)
        / (Sigma**2 * denom**2),
    ]

    dg[1, 1, [[1, 2]]] = [
        (-Delta * dSr + Sigma * dD) / Sigma**2,
        -Delta * dSt / Sigma**2,
    ]

    dg[2, 2, [[1, 2]]] = [-1.0 * dSr / Sigma**2, -1.0 * dSt / Sigma**2]

    dg[3, 3, [[1, 2]]] = [
        (
            (a**2 * np.sin(theta) ** 2 - Delta) * Sigma * ddmr
            + (a**2 * np.sin(theta) ** 2 - Delta) * dSr * denom
            + Sigma * dD * denom
        )
        / (Sigma**2 * denom**2),
        (
            -(a**2) * Sigma * denom * np.sin(2 * theta)
            + (a**2 * np.sin(theta) ** 2 - Delta) * Sigma * ddmt
            + (a**2 * np.sin(theta) ** 2 - Delta) * dSt * denom
        )
        / (Sigma**2 * denom**2),
    ]

    dg[0, 3, [[1, 2]]] = [
        2.0
        * M
        * a
        * (r * Sigma * ddmr + r * dSr * denom - Sigma * denom)
        * np.sin(theta) ** 2
        / (Sigma**2 * denom**2),
        M
        * a
        * r
        * (
            -1.0 * Sigma * ddmt * np.cos(2 * theta)
            + 1.0 * Sigma * ddmt
            - 2.0 * Sigma * denom * np.sin(2 * theta)
            - 1.0 * dSt * denom * np.cos(2 * theta)
            + 1.0 * dSt * denom
        )
        / (Sigma**2 * denom**2),
    ]

    dg[3, 0, [[1, 2]]] = dg[0, 3, [[1, 2]]]

    return dg  # return dg, [[1,2]]

In [ ]:
def Dkerr_christoffel(r, theta, a, M=1.0):
    """
    Function to give the christoffel symbols for kerr metric.
    The christoffel symbols are given as \Gamma ^i _{jk}

    From Reference Paper, Appendix
    Config: ull
    """

    dcs = np.zeros((4, 4, 4, 4), dtype=type(r))

    # Definitions
    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * np.cos(theta) ** 2
    dSt = -(a**2) * np.sin(2 * theta)
    dSr = 2 * r
    dD = 2 * (r - M)

    dcs[3, 0, 1, [[1, 2]]] = [
        M
        * a
        * (
            (4 * r - dSr) * Delta * Sigma
            - 2 * (2 * r**2 - Sigma) * Delta * dSr
            - (2 * r**2 - Sigma) * Sigma * dD
        )
        / (Delta**2 * Sigma**3),
        M * a * (-4 * r**2 + Sigma) * dSt / (Delta * Sigma**3),
    ]

    dcs[0, 0, 1, [[1, 2]]] = [
        M
        * (
            2 * r * (a**2 * np.sin(theta) ** 2 + 2 * r**2) * Delta * Sigma
            + 2 * (a**2 + r**2) * (a**2 * np.cos(theta) ** 2 - r**2) * Delta * dSr
            + (a**2 + r**2) * (a**2 * np.cos(theta) ** 2 - r**2) * Sigma * dD
        )
        / (Delta**2 * Sigma**3),
        2
        * M
        * (a**2 + r**2)
        * (
            (1 / 2) * a**2 * Sigma * np.sin(2 * theta)
            + (a**2 * np.cos(theta) ** 2 - r**2) * dSt
        )
        / (Delta * Sigma**3),
    ]

    dcs[3, 0, 2, [[1, 2]]] = [
        2 * M * a * (2 * r * dSr - Sigma) / (Sigma**3 * np.tan(theta)),
        2
        * M
        * a
        * r
        * (Sigma + dSt * np.sin(2 * theta))
        / (Sigma**3 * np.sin(theta) ** 2),
    ]

    dcs[0, 0, 2, [[1, 2]]] = [
        M * a**2 * (2 * r * dSr - Sigma) * np.sin(2 * theta) / Sigma**3,
        2
        * M
        * a**2
        * r
        * (-Sigma * np.cos(2 * theta) + dSt * np.sin(2 * theta))
        / Sigma**3,
    ]

    dcs[0, 1, 3, [[1, 2]]] = [
        M
        * a
        * (
            2 * (2 * r**2 * (a**2 + r**2) - (a**2 - r**2) * Sigma) * Delta * dSr
            + (2 * r**2 * (a**2 + r**2) - (a**2 - r**2) * Sigma) * Sigma * dD
            - (4 * r**3 + 4 * r * (a**2 + r**2) + 2 * r * Sigma - (a**2 - r**2) * dSr)
            * Delta
            * Sigma
        )
        * np.sin(theta) ** 2
        / (Delta**2 * Sigma**3),
        M
        * a
        * (
            2 * (2 * r**2 * (a**2 + r**2) - (a**2 - r**2) * Sigma) * dSt * np.sin(theta)
            + (
                (a**2 - r**2) * dSt * np.sin(theta)
                + (-4 * r**2 * (a**2 + r**2) + 2 * (a**2 - r**2) * Sigma)
                * np.cos(theta)
            )
            * Sigma
        )
        * np.sin(theta)
        / (Delta * Sigma**3),
    ]

    dcs[3, 1, 3, [[1, 2]]] = [
        (
            2
            * (
                M * a**2 * (2 * r**2 - Sigma) * np.sin(theta) ** 2
                + r * (2 * M * r - Sigma) * Sigma
            )
            * Delta
            * dSr
            + (
                M * a**2 * (2 * r**2 - Sigma) * np.sin(theta) ** 2
                + r * (2 * M * r - Sigma) * Sigma
            )
            * Sigma
            * dD
            - (
                M * a**2 * (4 * r - dSr) * np.sin(theta) ** 2
                + r * (2 * M - dSr) * Sigma
                + r * (2 * M * r - Sigma) * dSr
                + (2 * M * r - Sigma) * Sigma
            )
            * Delta
            * Sigma
        )
        / (Delta**2 * Sigma**3),
        M
        * (
            -2 * a**2 * r**2 * Sigma * np.sin(2 * theta)
            + 4 * a**2 * r**2 * dSt * np.sin(theta) ** 2
            + a**2 * Sigma**2 * np.sin(2 * theta)
            - a**2 * Sigma * dSt * np.sin(theta) ** 2
            + 2 * r**2 * Sigma * dSt
        )
        / (Delta * Sigma**3),
    ]

    dcs[0, 2, 3, [[1, 2]]] = [
        2
        * M
        * a**3
        * (-2 * r * dSr + Sigma)
        * np.sin(theta) ** 3
        * np.cos(theta)
        / Sigma**3,
        2
        * M
        * a**3
        * r
        * (Sigma * np.sin(3 * theta) - dSt * np.sin(theta) * np.sin(2 * theta))
        * np.sin(theta)
        / Sigma**3,
    ]

    dcs[3, 2, 3, [[1, 2]]] = [
        M * a**2 * (-2 * r * dSr + Sigma) * np.sin(2 * theta) / Sigma**3,
        2 * M * a**2 * r * np.cos(2 * theta) / Sigma**2
        - 2 * M * a**2 * r * dSt * np.sin(2 * theta) / Sigma**3
        + 2 / (np.cos(2 * theta) - 1),
    ]

    dcs[1, 0, 0, [[1, 2]]] = [
        M
        * (
            -3 * (2 * r**2 - Sigma) * Delta * dSr
            + ((4 * r - dSr) * Delta + (2 * r**2 - Sigma) * dD) * Sigma
        )
        / Sigma**4,
        2 * M * (-3 * r**2 + Sigma) * Delta * dSt / Sigma**4,
    ]

    dcs[1, 0, 3, [[1, 2]]] = [
        M
        * a
        * (
            3 * (2 * r**2 - Sigma) * Delta * dSr
            - ((4 * r - dSr) * Delta + (2 * r**2 - Sigma) * dD) * Sigma
        )
        * np.sin(theta) ** 2
        / Sigma**4,
        M
        * a
        * (
            3 * (2 * r**2 - Sigma) * dSt * np.sin(theta)
            + ((-4 * r**2 + 2 * Sigma) * np.cos(theta) + dSt * np.sin(theta)) * Sigma
        )
        * Delta
        * np.sin(theta)
        / Sigma**4,
    ]

    dcs[2, 0, 0, [[1, 2]]] = [
        M * a**2 * (3 * r * dSr - Sigma) * np.sin(2 * theta) / Sigma**4,
        M
        * a**2
        * r
        * (-2 * Sigma * np.cos(2 * theta) + 3 * dSt * np.sin(2 * theta))
        / Sigma**4,
    ]

    dcs[2, 0, 3, [[1, 2]]] = [
        (1 / 2)
        * M
        * a
        * (-6.0 * r * (a**2 + r**2) * dSr + (2.0 * a**2 + 6.0 * r**2) * Sigma)
        * np.sin(2 * theta)
        / Sigma**4,
        M
        * a
        * r
        * (a**2 + r**2)
        * (2.0 * Sigma * np.cos(2 * theta) - 3.0 * dSt * np.sin(2 * theta))
        / Sigma**4,
    ]

    dcs[1, 1, 1, [[1, 2]]] = [
        -M * dD / Delta**2
        - r * dSr / Sigma**2
        + r * dD / Delta**2
        + 1 / Sigma
        - 1 / Delta,
        -r * dSt / Sigma**2,
    ]

    dcs[1, 2, 2, [[1, 2]]] = [
        (r * Delta * dSr - (r * dD + Delta) * Sigma) / Sigma**2,
        r * Delta * dSt / Sigma**2,
    ]

    dcs[2, 1, 2, [[1, 2]]] = [(-r * dSr + Sigma) / Sigma**2, -r * dSt / Sigma**2]

    dcs[1, 1, 2, [[1, 2]]] = dcs[2, 2, 2, [[1, 2]]] = [
        (1 / 2) * a**2 * dSr * np.sin(2 * theta) / Sigma**2,
        a**2
        * (-Sigma * np.cos(2 * theta) + (1 / 2) * dSt * np.sin(2 * theta))
        / Sigma**2,
    ]

    dcs[2, 1, 1, [[1, 2]]] = [
        (1 / 2)
        * a**2
        * (-Delta * dSr - Sigma * dD)
        * np.sin(2 * theta)
        / (Delta**2 * Sigma**2),
        a**2
        * (Sigma * np.cos(2 * theta) - 1 / 2 * dSt * np.sin(2 * theta))
        / (Delta * Sigma**2),
    ]

    dcs[1, 3, 3, [[1, 2]]] = [
        (
            (
                (M * a**2 * (2 * r**2 - Sigma) * np.sin(theta) ** 2 - r * Sigma**2) * dD
                - (
                    -M * a**2 * (4 * r - dSr) * np.sin(theta) ** 2
                    + 2 * r * Sigma * dSr
                    + Sigma**2
                )
                * Delta
            )
            * Sigma
            - 3
            * (M * a**2 * (2 * r**2 - Sigma) * np.sin(theta) ** 2 - r * Sigma**2)
            * Delta
            * dSr
        )
        * np.sin(theta) ** 2
        / Sigma**4,
        (
            -3 / 2 * M * a**2 * r**2 * (1 - np.cos(2 * theta)) ** 2 * dSt
            + 2 * M * a**2 * r**2 * Sigma * np.sin(2 * theta)
            - M * a**2 * r**2 * Sigma * np.sin(4 * theta)
            + (1 / 2) * M * a**2 * (1 - np.cos(2 * theta)) ** 2 * Sigma * dSt
            - M * a**2 * Sigma**2 * np.sin(2 * theta)
            + (1 / 2) * M * a**2 * Sigma**2 * np.sin(4 * theta)
            - r * Sigma**3 * np.sin(2 * theta)
            - 1 / 2 * r * Sigma**2 * dSt * np.cos(2 * theta)
            + (1 / 2) * r * Sigma**2 * dSt
        )
        * Delta
        / Sigma**4,
    ]

    dcs[2, 3, 3, [[1, 2]]] = [
        (1 / 2)
        * (
            3 * (2 * M * r * (a**2 + r**2) ** 2 + Delta * Sigma**2) * dSr
            - (
                8 * M * r**2 * (a**2 + r**2)
                + 2 * M * (a**2 + r**2) ** 2
                + 2 * Delta * Sigma * dSr
                + Sigma**2 * dD
            )
            * Sigma
        )
        * np.sin(2 * theta)
        / Sigma**4,
        -2 * M * r * (a**2 + r**2) ** 2 * np.cos(2 * theta) / Sigma**3
        + 3 * M * r * (a**2 + r**2) ** 2 * dSt * np.sin(2 * theta) / Sigma**4
        - Delta * np.cos(2 * theta) / Sigma
        + (1 / 2) * Delta * dSt * np.sin(2 * theta) / Sigma**2,
    ]

    dcs[0, 1, 0, [[1, 2]]] = dcs[0, 0, 1, [[1, 2]]]
    dcs[0, 2, 0, [[1, 2]]] = dcs[0, 0, 2, [[1, 2]]]
    dcs[0, 3, 1, [[1, 2]]] = dcs[0, 1, 3, [[1, 2]]]
    dcs[0, 3, 2, [[1, 2]]] = dcs[0, 2, 3, [[1, 2]]]

    dcs[1, 3, 0, [[1, 2]]] = dcs[1, 0, 3, [[1, 2]]]
    dcs[1, 2, 1, [[1, 2]]] = dcs[1, 1, 2, [[1, 2]]]

    dcs[2, 3, 0, [[1, 2]]] = dcs[2, 0, 3, [[1, 2]]]
    dcs[2, 2, 1, [[1, 2]]] = dcs[2, 1, 2, [[1, 2]]]

    dcs[3, 1, 0, [[1, 2]]] = dcs[3, 0, 1, [[1, 2]]]
    dcs[3, 2, 0, [[1, 2]]] = dcs[3, 0, 2, [[1, 2]]]
    dcs[3, 3, 1, [[1, 2]]] = dcs[3, 1, 3, [[1, 2]]]
    dcs[3, 3, 2, [[1, 2]]] = dcs[3, 2, 3, [[1, 2]]]

    # return symmetrize(arr=dcs)+np.zer, [[1,2]]os((4,4,4))
    return dcs + np.zeros((4, 4, 4, 4))
    # return dcs, [[1,2]]

In [ ]:
def DRijkl(r,theta,a, M=1.0, config="ullll"):

    drijkl= np.zeros((4,4,4,4,4),dtype=type(r))

    Delta=  a**2 - 2 * M * r + r**2
    Sigma =  r**2 + a**2 * np.cos(theta) ** 2
    dSt= -a**2*np.sin(2*theta)
    dSr=  2*r
    dD=  2*(r-M)

    drijkl[0, 0, 0, 3, :] = [0, M**2*a*r*(8.0*r*(3.0*a**2*np.cos(theta)**2 - r**2)*dSr + 4.0*(-3.0*a**2*np.cos(theta)**2 + 2.0*r**2)*Sigma)*np.sin(theta)**2/Sigma**5, M**2*a*r**2*(-3.0*a**2*Sigma*np.sin(4*theta) - 3.0*a**2*dSt*np.cos(4*theta) + 3.0*a**2*dSt + 2.0*r**2*Sigma*np.sin(2*theta) + 4.0*r**2*dSt*np.cos(2*theta) - 4.0*r**2*dSt)/Sigma**5, 0.]

    drijkl[0, 0, 1, 2, :] = [0, M**2*a**2*(-6.0*r*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta*dSr - 2.0*r*(a**2*np.cos(theta)**2 - 3.0*r**2)*Sigma*dD + (2.0*a**2*np.cos(theta)**2 - 18.0*r**2)*Delta*Sigma)*np.sin(theta)*np.cos(theta)/(Delta**2*Sigma**4), M**2*a**2*r*(-6.0*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (8.0*a**2*np.sin(theta)**4 - 10.0*a**2*np.sin(theta)**2 + 2.0*a**2 + 12.0*r**2*np.sin(theta)**2 - 6.0*r**2)*Sigma)/(Delta*Sigma**4), 0.]

    drijkl[0, 1, 0, 1, :] = [0, M*(3*r*(3.0*a**2*np.cos(theta)**2 - r**2)*(a**2*np.sin(theta)**2 + 2.0*a**2 + 2.0*r**2)*Delta*dSr + r*(3.0*a**2*np.cos(theta)**2 - r**2)*(a**2*np.sin(theta)**2 + 2.0*a**2 + 2.0*r**2)*Sigma*dD + (3.0*a**4*np.sin(theta)**4 + 3.0*a**4*np.sin(theta)**2 - 6.0*a**4 + 21.0*a**2*r**2*np.sin(theta)**2 - 12.0*a**2*r**2 + 10.0*r**4)*Delta*Sigma)/(Delta**2*Sigma**4), M*r*(a**2*(12.0*a**2*np.sin(theta)**2 + 6.0*a**2 + 14.0*r**2)*Sigma*np.sin(theta)*np.cos(theta) + 3*(3.0*a**2*np.cos(theta)**2 - r**2)*(a**2*np.sin(theta)**2 + 2.0*a**2 + 2.0*r**2)*dSt)/(Delta*Sigma**4), 0.]

    drijkl[0, 1, 0, 2, :] = [0, M*a**2*(-3*(a**2*np.cos(theta)**2 - 3.0*r**2)*(-2*M*r + 3.0*a**2 + 3.0*r**2)*Delta*dSr - (a**2*np.cos(theta)**2 - 3.0*r**2)*(-2*M*r + 3.0*a**2 + 3.0*r**2)*Sigma*dD + (-6.0*r*(-2*M*r + 3.0*a**2 + 3.0*r**2) + (-2*M + 6.0*r)*(a**2*np.cos(theta)**2 - 3.0*r**2))*Delta*Sigma)*np.sin(theta)*np.cos(theta)/(Delta**2*Sigma**4), M*a**2*(-3*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (4.0*a**2*np.sin(theta)**4 - 5.0*a**2*np.sin(theta)**2 + 1.0*a**2 + 6.0*r**2*np.sin(theta)**2 - 3.0*r**2)*Sigma)*(-2*M*r + 3.0*a**2 + 3.0*r**2)/(Delta*Sigma**4), 0.]

    drijkl[0, 1, 1, 3, :] = [0, M*a*(9.0*r*(a**2 + r**2)*(3.0*a**2*np.cos(theta)**2 - r**2)*Delta*dSr + 3.0*r*(a**2 + r**2)*(3.0*a**2*np.cos(theta)**2 - r**2)*Sigma*dD + (-9.0*a**4*np.cos(theta)**2 - 27.0*a**2*r**2*np.cos(theta)**2 + 9.0*a**2*r**2 + 15.0*r**4)*Delta*Sigma)*np.sin(theta)**2/(Delta**2*Sigma**4), M*a*r*(a**2 + r**2)*(-4.5*a**2*Sigma*np.sin(4*theta) - 3.375*a**2*dSt*np.cos(4*theta) + 3.375*a**2*dSt + 3.0*r**2*Sigma*np.sin(2*theta) + 4.5*r**2*dSt*np.cos(2*theta) - 4.5*r**2*dSt)/(Delta*Sigma**4), 0.]

    drijkl[0, 1, 2, 3, :] = [0., M*a*(-3*(a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*Delta*np.sin(theta)**2 + 2.0*(a**2 + r**2)**2)*Delta*dSr - (a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*Delta*np.sin(theta)**2 + 2.0*(a**2 + r**2)**2)*Sigma*dD + (-6.0*r*(a**2*Delta*np.sin(theta)**2 + 2.0*(a**2 + r**2)**2) + (a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*dD*np.sin(theta)**2 + 8.0*r*(a**2 + r**2)))*Delta*Sigma)*np.sin(theta)*np.cos(theta)/(Delta**2*Sigma**4), M*a*(2*a**2*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta*Sigma*np.sin(theta)**2*np.cos(theta)**2 - 3*(a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*Delta*np.sin(theta)**2 + 2.0*(a**2 + r**2)**2)*dSt*np.sin(theta)*np.cos(theta) + (a**2*Delta*np.sin(theta)**2 + 2.0*(a**2 + r**2)**2)*(4.0*a**2*np.sin(theta)**4 - 5.0*a**2*np.sin(theta)**2 + 1.0*a**2 + 6.0*r**2*np.sin(theta)**2 - 3.0*r**2)*Sigma)/(Delta*Sigma**4), 0.]

    # Antisymmetric under exchange of last two indices.
    drijkl[0, 0, 3, 0, :] = -drijkl[0, 0, 0, 3, :]
    drijkl[0, 0, 2, 1, :] = -drijkl[0, 0, 1, 2, :]
    drijkl[0, 1, 1, 0, :] = -drijkl[0, 1, 0, 1, :]
    drijkl[0, 1, 2, 0, :] = -drijkl[0, 1, 0, 2, :]
    drijkl[0, 1, 3, 1, :] = -drijkl[0, 1, 1, 3, :]
    drijkl[0, 1, 3, 2, :] = -drijkl[0, 1, 2, 3, :]

    #! -------------------------------------------------------------------------------------#

    drijkl[0, 2, 0, 1, :] = [0., M*a**2*(-3*(a**2*np.cos(theta)**2 - 3.0*r**2)*(-4*M*r + 3.0*a**2 + 3.0*r**2)*Delta*dSr - (a**2*np.cos(theta)**2 - 3.0*r**2)*(-4*M*r + 3.0*a**2 + 3.0*r**2)*Sigma*dD + (-6.0*r*(-4*M*r + 3.0*a**2 + 3.0*r**2) + (-4*M + 6.0*r)*(a**2*np.cos(theta)**2 - 3.0*r**2))*Delta*Sigma)*np.sin(theta)*np.cos(theta)/(Delta**2*Sigma**4), M*a**2*(-3*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (4.0*a**2*np.sin(theta)**4 - 5.0*a**2*np.sin(theta)**2 + 1.0*a**2 + 6.0*r**2*np.sin(theta)**2 - 3.0*r**2)*Sigma)*(-4*M*r + 3.0*a**2 + 3.0*r**2)/(Delta*Sigma**4), 0.]

    drijkl[0, 2, 0, 2, :] = [0., M*(-3*r*(3.0*a**2*np.cos(theta)**2 - r**2)*(2*a**2*np.sin(theta)**2 + a**2 + r**2)*dSr + (-6.0*a**4*np.sin(theta)**4 + 3.0*a**4*np.sin(theta)**2 + 3.0*a**4 - 15.0*a**2*r**2*np.sin(theta)**2 + 6.0*a**2*r**2 - 5.0*r**4)*Sigma)/Sigma**4, M*r*(a**2*(-24.0*a**2*np.sin(theta)**2 + 6.0*a**2 - 10.0*r**2)*Sigma*np.sin(theta)*np.cos(theta) - 3*(3.0*a**2*np.cos(theta)**2 - r**2)*(2*a**2*np.sin(theta)**2 + a**2 + r**2)*dSt)/Sigma**4, 0.]

    drijkl[0, 2, 1, 3, :] = [0., M*a*(-3*(a**2*np.cos(theta)**2 - 3.0*r**2)*(2.0*a**2*Delta*np.sin(theta)**2 + (a**2 + r**2)**2)*Delta*dSr - (a**2*np.cos(theta)**2 - 3.0*r**2)*(2.0*a**2*Delta*np.sin(theta)**2 + (a**2 + r**2)**2)*Sigma*dD + (-6.0*r*(2.0*a**2*Delta*np.sin(theta)**2 + (a**2 + r**2)**2) + (a**2*np.cos(theta)**2 - 3.0*r**2)*(2.0*a**2*dD*np.sin(theta)**2 + 4*r*(a**2 + r**2)))*Delta*Sigma)*np.sin(theta)*np.cos(theta)/(Delta**2*Sigma**4), M*a*(4.0*a**2*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta*Sigma*np.sin(theta)**2*np.cos(theta)**2 - 3*(a**2*np.cos(theta)**2 - 3.0*r**2)*(2.0*a**2*Delta*np.sin(theta)**2 + (a**2 + r**2)**2)*dSt*np.sin(theta)*np.cos(theta) + (2.0*a**2*Delta*np.sin(theta)**2 + (a**2 + r**2)**2)*(4.0*a**2*np.sin(theta)**4 - 5.0*a**2*np.sin(theta)**2 + 1.0*a**2 + 6.0*r**2*np.sin(theta)**2 - 3.0*r**2)*Sigma)/(Delta*Sigma**4), 0.]

    drijkl[0, 2, 2, 3, :] = [0., M*a*(-9.0*r*(a**2 + r**2)*(3.0*a**2*np.cos(theta)**2 - r**2)*dSr + (9.0*a**4*np.cos(theta)**2 + 27.0*a**2*r**2*np.cos(theta)**2 - 9.0*a**2*r**2 - 15.0*r**4)*Sigma)*np.sin(theta)**2/Sigma**4, M*a*r*(a**2 + r**2)*(4.5*a**2*Sigma*np.sin(4*theta) + 3.375*a**2*dSt*np.cos(4*theta) - 3.375*a**2*dSt - 3.0*r**2*Sigma*np.sin(2*theta) - 4.5*r**2*dSt*np.cos(2*theta) + 4.5*r**2*dSt)/Sigma**4, 0.]

    drijkl[0, 3, 0, 3, :] = [0., M*(-4*r*(-3*a**2*np.cos(theta)**2 + r**2)*(a**2*Delta*np.sin(theta)**2 - (a**2 + r**2)**2)*dSr + (2*r**2*(a**2*Delta*np.sin(theta)**2 - (a**2 + r**2)**2) + r*(-3*a**2*np.cos(theta)**2 + r**2)*(a**2*dD*np.sin(theta)**2 - 4*r*(a**2 + r**2)) + (-3*a**2*np.cos(theta)**2 + r**2)*(a**2*Delta*np.sin(theta)**2 - (a**2 + r**2)**2))*Sigma)*np.sin(theta)**2/Sigma**5, 2*M*r*(-2*(-3*a**2*np.cos(theta)**2 + r**2)*(a**2*Delta*np.sin(theta)**2 - (a**2 + r**2)**2)*dSt*np.sin(theta) + (a**2*(-3*a**2*np.cos(theta)**2 + r**2)*Delta*np.sin(theta)**2 + 3*a**2*(a**2*Delta*np.sin(theta)**2 - (a**2 + r**2)**2)*np.sin(theta)**2 + (-3*a**2*np.cos(theta)**2 + r**2)*(a**2*Delta*np.sin(theta)**2 - (a**2 + r**2)**2))*Sigma*np.cos(theta))*np.sin(theta)/Sigma**5, 0.]

    drijkl[0, 3, 1, 2, :] = [0., M*a*(-3*(a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*Delta*np.sin(theta)**2 - (a**2 + r**2)**2)*Delta*dSr - (a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*Delta*np.sin(theta)**2 - (a**2 + r**2)**2)*Sigma*dD + (-6.0*r*(a**2*Delta*np.sin(theta)**2 - (a**2 + r**2)**2) + (a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*dD*np.sin(theta)**2 - 4*r*(a**2 + r**2)))*Delta*Sigma)*np.sin(theta)*np.cos(theta)/(Delta**2*Sigma**4), M*a*(2*a**2*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta*Sigma*np.sin(theta)**2*np.cos(theta)**2 - 3*(a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*Delta*np.sin(theta)**2 - (a**2 + r**2)**2)*dSt*np.sin(theta)*np.cos(theta) + (a**2*Delta*np.sin(theta)**2 - (a**2 + r**2)**2)*(4.0*a**2*np.sin(theta)**4 - 5.0*a**2*np.sin(theta)**2 + 1.0*a**2 + 6.0*r**2*np.sin(theta)**2 - 3.0*r**2)*Sigma)/(Delta*Sigma**4), 0.]

    # Symmetries
    drijkl[0, 2, 1, 0, :] = -drijkl[0, 2, 0, 1, :]
    drijkl[0, 2, 2, 0, :] = -drijkl[0, 2, 0, 2, :]
    drijkl[0, 2, 3, 1, :] = -drijkl[0, 2, 1, 3, :]
    drijkl[0, 2, 3, 2, :] = -drijkl[0, 2, 2, 3, :]
    drijkl[0, 3, 3, 0, :] = -drijkl[0, 3, 0, 3, :]
    drijkl[0, 3, 2, 1, :] = -drijkl[0, 3, 1, 2, :]

    #! ------------------------------------------------------------------------------#

    drijkl[1, 0, 0, 1, :] = [0., M*(4*r*(a**2*np.sin(theta)**2 + 2.0*Delta)*(3.0*a**2*np.cos(theta)**2 - r**2)*dSr + (2*r**2*(a**2*np.sin(theta)**2 + 2.0*Delta) - 2.0*r*(3.0*a**2*np.cos(theta)**2 - r**2)*dD - (a**2*np.sin(theta)**2 + 2.0*Delta)*(3.0*a**2*np.cos(theta)**2 - r**2))*Sigma)/Sigma**5, M*r*(a**2*(12.0*a**2*np.sin(theta)**2 - 6.0*a**2 + 2.0*r**2 + 12.0*Delta)*Sigma*np.sin(theta)*np.cos(theta) + 4*(a**2*np.sin(theta)**2 + 2.0*Delta)*(3.0*a**2*np.cos(theta)**2 - r**2)*dSt)/Sigma**5, 0.]

    drijkl[1, 0, 0, 2, :] = [0., M*a**2*(-12.0*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta*dSr + (-18.0*r*Delta + 3.0*(a**2*np.cos(theta)**2 - 3.0*r**2)*dD)*Sigma)*np.sin(theta)*np.cos(theta)/Sigma**5, M*a**2*(-12.0*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (12.0*a**2*np.sin(theta)**4 - 15.0*a**2*np.sin(theta)**2 + 3.0*a**2 + 18.0*r**2*np.sin(theta)**2 - 9.0*r**2)*Sigma)*Delta/Sigma**5, 0.]

    drijkl[1, 0, 1, 3, :] = [0., M*a*(4*r*(3.0*a**2*np.cos(theta)**2 - r**2)*(-4.0*M*r + 3.0*a**2 + 3.0*r**2)*dSr + (r**2*(-8.0*M*r + 6.0*a**2 + 6.0*r**2) + r*(4.0*M - 6.0*r)*(3.0*a**2*np.cos(theta)**2 - r**2) + (3.0*a**2*np.cos(theta)**2 - r**2)*(4.0*M*r - 3.0*a**2 - 3.0*r**2))*Sigma)*np.sin(theta)**2/Sigma**5, M*a*r*(-4.0*M*r + 3.0*a**2 + 3.0*r**2)*(-1.5*a**2*Sigma*np.sin(4*theta) - 1.5*a**2*dSt*np.cos(4*theta) + 1.5*a**2*dSt + r**2*Sigma*np.sin(2*theta) + 2.0*r**2*dSt*np.cos(2*theta) - 2.0*r**2*dSt)/Sigma**5, 0.]

    drijkl[1, 0, 2, 3, :] = [0., M*a*(-4*(a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*np.sin(theta)**2 + 2.0*a**2 + 2.0*r**2)*Delta*dSr + (4.0*r*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta - 6.0*r*(a**2*np.sin(theta)**2 + 2.0*a**2 + 2.0*r**2)*Delta + (a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*np.sin(theta)**2 + 2.0*a**2 + 2.0*r**2)*dD)*Sigma)*np.sin(theta)*np.cos(theta)/Sigma**5, M*a*(-4*(a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*np.sin(theta)**2 + 2.0*a**2 + 2.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (6.0*a**4*np.sin(theta)**6 - 1.0*a**4*np.sin(theta)**4 - 7.0*a**4*np.sin(theta)**2 + 2.0*a**4 + 20.0*a**2*r**2*np.sin(theta)**4 - 7.0*a**2*r**2*np.sin(theta)**2 - 4.0*a**2*r**2 + 12.0*r**4*np.sin(theta)**2 - 6.0*r**4)*Sigma)*Delta/Sigma**5, 0.]

    drijkl[1, 2, 0, 3, :] = [0., M*a*(-3*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta*dSr + (-6.0*r*Delta + (a**2*np.cos(theta)**2 - 3.0*r**2)*dD)*Sigma)*np.sin(theta)*np.cos(theta)/Sigma**4, M*a*(-3*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (4.0*a**2*np.sin(theta)**4 - 5.0*a**2*np.sin(theta)**2 + 1.0*a**2 + 6.0*r**2*np.sin(theta)**2 - 3.0*r**2)*Sigma)*Delta/Sigma**4, 0.]

    drijkl[1, 2, 1, 2, :] = [0., M*(-2*r*(3.0*a**2*np.cos(theta)**2 - r**2)*dSr + 3.0*(a**2*np.cos(theta)**2 - r**2)*Sigma)/Sigma**3, M*r*(-3.0*a**2*Sigma*np.sin(2*theta) - 2*(3.0*a**2*np.cos(theta)**2 - r**2)*dSt)/Sigma**3, 0.]

    drijkl[1, 3, 0, 1, :] = [0., M*a*(-4*r*(3.0*a**2*np.cos(theta)**2 - r**2)*(-4.0*M*r + 3.0*a**2 + 3.0*r**2)*dSr + (r**2*(8.0*M*r - 6.0*a**2 - 6.0*r**2) - r*(4.0*M - 6.0*r)*(3.0*a**2*np.cos(theta)**2 - r**2) + (3.0*a**2*np.cos(theta)**2 - r**2)*(-4.0*M*r + 3.0*a**2 + 3.0*r**2))*Sigma)*np.sin(theta)**2/Sigma**5, M*a*r*(-4.0*M*r + 3.0*a**2 + 3.0*r**2)*(1.5*a**2*Sigma*np.sin(4*theta) + 1.5*a**2*dSt*np.cos(4*theta) - 1.5*a**2*dSt - 1.0*r**2*Sigma*np.sin(2*theta) - 2.0*r**2*dSt*np.cos(2*theta) + 2.0*r**2*dSt)/Sigma**5, 0.]

    drijkl[1, 3, 0, 2, :] = [0., M*a*(4*(a**2*np.cos(theta)**2 - 3.0*r**2)*(2.0*a**2*np.sin(theta)**2 + a**2 + r**2)*Delta*dSr + (-2*r*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta + 6.0*r*(2.0*a**2*np.sin(theta)**2 + a**2 + r**2)*Delta - (a**2*np.cos(theta)**2 - 3.0*r**2)*(2.0*a**2*np.sin(theta)**2 + a**2 + r**2)*dD)*Sigma)*np.sin(theta)*np.cos(theta)/Sigma**5, M*a*(4*(a**2*np.cos(theta)**2 - 3.0*r**2)*(2.0*a**2*np.sin(theta)**2 + a**2 + r**2)*dSt*np.sin(theta)*np.cos(theta) + (-12.0*a**4*np.sin(theta)**6 + 14.0*a**4*np.sin(theta)**4 - 1.0*a**4*np.sin(theta)**2 - 1.0*a**4 - 28.0*a**2*r**2*np.sin(theta)**4 + 17.0*a**2*r**2*np.sin(theta)**2 + 2.0*a**2*r**2 - 6.0*r**4*np.sin(theta)**2 + 3.0*r**4)*Sigma)*Delta/Sigma**5, 0.]

    drijkl[1, 3, 1, 3, :] = [0., M*(-4*r*(3.0*a**2*np.cos(theta)**2 - r**2)*(2.0*a**2*Delta*np.sin(theta)**2 + (a**2 + r**2)**2)*dSr + (-2*r**2*(2.0*a**2*Delta*np.sin(theta)**2 + (a**2 + r**2)**2) + r*(3.0*a**2*np.cos(theta)**2 - r**2)*(2.0*a**2*dD*np.sin(theta)**2 + 4*r*(a**2 + r**2)) + (3.0*a**2*np.cos(theta)**2 - r**2)*(2.0*a**2*Delta*np.sin(theta)**2 + (a**2 + r**2)**2))*Sigma)*np.sin(theta)**2/Sigma**5, M*r*(-4*(3.0*a**2*np.cos(theta)**2 - r**2)*(2.0*a**2*Delta*np.sin(theta)**2 + (a**2 + r**2)**2)*dSt*np.sin(theta) + (4.0*a**2*(3.0*a**2*np.cos(theta)**2 - r**2)*Delta*np.sin(theta)**2 - 6.0*a**2*(2.0*a**2*Delta*np.sin(theta)**2 + (a**2 + r**2)**2)*np.sin(theta)**2 + 2*(3.0*a**2*np.cos(theta)**2 - r**2)*(2.0*a**2*Delta*np.sin(theta)**2 + (a**2 + r**2)**2))*Sigma*np.cos(theta))*np.sin(theta)/Sigma**5, 0.]

    drijkl[1, 3, 2, 3, :] = [0., M*a**2*(12.0*(a**2 + r**2)*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta*dSr + (18.0*r*(a**2 + r**2)*Delta - 6.0*r*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta - 3.0*(a**2 + r**2)*(a**2*np.cos(theta)**2 - 3.0*r**2)*dD)*Sigma)*np.sin(theta)**3*np.cos(theta)/Sigma**5, M*a**2*(a**2 + r**2)*(12.0*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (-18.0*a**2*np.sin(theta)**4 + 27.0*a**2*np.sin(theta)**2 - 9.0*a**2 - 36.0*r**2*np.sin(theta)**2 + 27.0*r**2)*Sigma)*Delta*np.sin(theta)**2/Sigma**5, 0.]

    # Symmetries
    drijkl[1, 0, 1, 0, :] = -drijkl[1, 0, 0, 1, :]
    drijkl[1, 0, 2, 0, :] = -drijkl[1, 0, 0, 2, :]
    drijkl[1, 0, 3, 1, :] = -drijkl[1, 0, 1, 3, :]
    drijkl[1, 0, 3, 2, :] = -drijkl[1, 0, 2, 3, :]
    drijkl[1, 2, 3, 0, :] = -drijkl[1, 2, 0, 3, :]
    drijkl[1, 2, 2, 1, :] = -drijkl[1, 2, 1, 2, :]
    drijkl[1, 3, 1, 0, :] = -drijkl[1, 3, 0, 1, :]
    drijkl[1, 3, 2, 0, :] = -drijkl[1, 3, 0, 2, :]
    drijkl[1, 3, 3, 1, :] = -drijkl[1, 3, 1, 3, :]
    drijkl[1, 3, 3, 2, :] = -drijkl[1, 3, 2, 3, :]

    #! -------------------------------------------------------------------------------------#

    drijkl[2, 0, 0, 1, :] = [0., M*a**2*(-18.0*r*Sigma - 12.0*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSr)*np.sin(theta)*np.cos(theta)/Sigma**5, M*a**2*(-12.0*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (12.0*a**2*np.sin(theta)**4 - 15.0*a**2*np.sin(theta)**2 + 3.0*a**2 + 18.0*r**2*np.sin(theta)**2 - 9.0*r**2)*Sigma)/Sigma**5, 0.] 

    drijkl[2, 0, 0, 2, :] = [0., M*(-4*r*(2.0*a**2*np.sin(theta)**2 + Delta)*(3.0*a**2*np.cos(theta)**2 - r**2)*dSr + (-2*r**2*(2.0*a**2*np.sin(theta)**2 + Delta) + r*(3.0*a**2*np.cos(theta)**2 - r**2)*dD + (2.0*a**2*np.sin(theta)**2 + Delta)*(3.0*a**2*np.cos(theta)**2 - r**2))*Sigma)/Sigma**5, M*r*(a**2*(-24.0*a**2*np.sin(theta)**2 + 12.0*a**2 - 4.0*r**2 - 6.0*Delta)*Sigma*np.sin(theta)*np.cos(theta) - 4*(2.0*a**2*np.sin(theta)**2 + Delta)*(3.0*a**2*np.cos(theta)**2 - r**2)*dSt)/Sigma**5, 0.]

    drijkl[2, 0, 1, 3, :] = [0., M*a*(r*(14.0*a**2*np.cos(theta)**2 - 18.0*a**2 - 12.0*r**2)*Sigma - 4*(a**2*np.cos(theta)**2 - 3.0*r**2)*(2.0*a**2*np.sin(theta)**2 + a**2 + r**2)*dSr)*np.sin(theta)*np.cos(theta)/Sigma**5, M*a*(-4*(a**2*np.cos(theta)**2 - 3.0*r**2)*(2.0*a**2*np.sin(theta)**2 + a**2 + r**2)*dSt*np.sin(theta)*np.cos(theta) + (12.0*a**4*np.sin(theta)**6 - 14.0*a**4*np.sin(theta)**4 + 1.0*a**4*np.sin(theta)**2 + 1.0*a**4 + 28.0*a**2*r**2*np.sin(theta)**4 - 17.0*a**2*r**2*np.sin(theta)**2 - 2.0*a**2*r**2 + 6.0*r**4*np.sin(theta)**2 - 3.0*r**4)*Sigma)/Sigma**5, 0.]

    drijkl[2, 0, 2, 3, :] = [0., M*a*(-4*r*(3.0*a**2*np.cos(theta)**2 - r**2)*(-2.0*M*r + 3.0*a**2 + 3.0*r**2)*dSr + (r**2*(4.0*M*r - 6.0*a**2 - 6.0*r**2) - r*(2.0*M - 6.0*r)*(3.0*a**2*np.cos(theta)**2 - r**2) + (3.0*a**2*np.cos(theta)**2 - r**2)*(-2.0*M*r + 3.0*a**2 + 3.0*r**2))*Sigma)*np.sin(theta)**2/Sigma**5, M*a*r*(-2.0*M*r + 3.0*a**2 + 3.0*r**2)*(1.5*a**2*Sigma*np.sin(4*theta) + 1.5*a**2*dSt*np.cos(4*theta) - 1.5*a**2*dSt - 1.0*r**2*Sigma*np.sin(2*theta) - 2.0*r**2*dSt*np.cos(2*theta) + 2.0*r**2*dSt)/Sigma**5, 0.]

    drijkl[2, 1, 0, 3, :] = [0., M*a*(6.0*r*Sigma + 3*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSr)*np.sin(theta)*np.cos(theta)/Sigma**4, M*a*(3*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (-4.0*a**2*np.sin(theta)**4 + 5.0*a**2*np.sin(theta)**2 - 1.0*a**2 - 6.0*r**2*np.sin(theta)**2 + 3.0*r**2)*Sigma)/Sigma**4, 0.]

    drijkl[2, 1, 1, 2, :] = [0., M*(2*r*(3.0*a**2*np.cos(theta)**2 - r**2)*Delta*dSr + r*(3.0*a**2*np.cos(theta)**2 - r**2)*Sigma*dD + 3.0*(-a**2*np.cos(theta)**2 + r**2)*Delta*Sigma)/(Delta**2*Sigma**3), M*r*(3.0*a**2*Sigma*np.sin(2*theta) + 2*(3.0*a**2*np.cos(theta)**2 - r**2)*dSt)/(Delta*Sigma**3), 0.]

    drijkl[2, 3, 0, 1, :] = [0., M*a*(r*(10.0*a**2*np.sin(theta)**2 + 8.0*a**2 + 24.0*r**2)*Sigma + 4*(a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*np.sin(theta)**2 + 2.0*a**2 + 2.0*r**2)*dSr)*np.sin(theta)*np.cos(theta)/Sigma**5, M*a*(4*(a**2*np.cos(theta)**2 - 3.0*r**2)*(a**2*np.sin(theta)**2 + 2.0*a**2 + 2.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (-6.0*a**4*np.sin(theta)**6 + 1.0*a**4*np.sin(theta)**4 + 7.0*a**4*np.sin(theta)**2 - 2.0*a**4 - 20.0*a**2*r**2*np.sin(theta)**4 + 7.0*a**2*r**2*np.sin(theta)**2 + 4.0*a**2*r**2 - 12.0*r**4*np.sin(theta)**2 + 6.0*r**4)*Sigma)/Sigma**5, 0.]

    drijkl[2, 3, 0, 2, :] = [0., M*a*(4*r*(3.0*a**2*np.cos(theta)**2 - r**2)*(-2.0*M*r + 3.0*a**2 + 3.0*r**2)*dSr + (r**2*(-4.0*M*r + 6.0*a**2 + 6.0*r**2) + r*(2.0*M - 6.0*r)*(3.0*a**2*np.cos(theta)**2 - r**2) + (3.0*a**2*np.cos(theta)**2 - r**2)*(2.0*M*r - 3.0*a**2 - 3.0*r**2))*Sigma)*np.sin(theta)**2/Sigma**5, M*a*r*(-2.0*M*r + 3.0*a**2 + 3.0*r**2)*(-1.5*a**2*Sigma*np.sin(4*theta) - 1.5*a**2*dSt*np.cos(4*theta) + 1.5*a**2*dSt + r**2*Sigma*np.sin(2*theta) + 2.0*r**2*dSt*np.cos(2*theta) - 2.0*r**2*dSt)/Sigma**5, 0.]

    drijkl[2, 3, 1, 3, :] = [0., M*a**2*(r*(-6.0*a**2*np.cos(theta)**2 + 18.0*a**2 + 36.0*r**2)*Sigma + 12.0*(a**2 + r**2)*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSr)*np.sin(theta)**3*np.cos(theta)/Sigma**5, M*a**2*(a**2 + r**2)*(12.0*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (-18.0*a**2*np.sin(theta)**4 + 27.0*a**2*np.sin(theta)**2 - 9.0*a**2 - 36.0*r**2*np.sin(theta)**2 + 27.0*r**2)*Sigma)*np.sin(theta)**2/Sigma**5, 0.]

    drijkl[2, 3, 2, 3, :] = [0., M*(4*r*(3.0*a**2*np.cos(theta)**2 - r**2)*(a**2*Delta*np.sin(theta)**2 + 2.0*(a**2 + r**2)**2)*dSr + (2*r**2*(a**2*Delta*np.sin(theta)**2 + 2.0*(a**2 + r**2)**2) - r*(3.0*a**2*np.cos(theta)**2 - r**2)*(a**2*dD*np.sin(theta)**2 + 8.0*r*(a**2 + r**2)) - (3.0*a**2*np.cos(theta)**2 - r**2)*(a**2*Delta*np.sin(theta)**2 + 2.0*(a**2 + r**2)**2))*Sigma)*np.sin(theta)**2/Sigma**5, M*r*(4*(3.0*a**2*np.cos(theta)**2 - r**2)*(a**2*Delta*np.sin(theta)**2 + 2.0*(a**2 + r**2)**2)*dSt*np.sin(theta) + (-2*a**2*(3.0*a**2*np.cos(theta)**2 - r**2)*Delta*np.sin(theta)**2 + 6.0*a**2*(a**2*Delta*np.sin(theta)**2 + 2.0*(a**2 + r**2)**2)*np.sin(theta)**2 - 2*(3.0*a**2*np.cos(theta)**2 - r**2)*(a**2*Delta*np.sin(theta)**2 + 2.0*(a**2 + r**2)**2))*Sigma*np.cos(theta))*np.sin(theta)/Sigma**5, 0.]

    # Symmetries
    drijkl[2, 0, 1, 0, :] = -drijkl[2, 0, 0, 1, :]
    drijkl[2, 0, 2, 0, :] = -drijkl[2, 0, 0, 2, :]
    drijkl[2, 0, 3, 1, :] = -drijkl[2, 0, 1, 3, :]
    drijkl[2, 0, 3, 2, :] = -drijkl[2, 0, 2, 3, :]
    drijkl[2, 1, 3, 0, :] = -drijkl[2, 1, 0, 3, :]
    drijkl[2, 1, 2, 1, :] = -drijkl[2, 1, 1, 2, :]
    drijkl[2, 3, 1, 0, :] = -drijkl[2, 3, 0, 1, :]
    drijkl[2, 3, 2, 0, :] = -drijkl[2, 3, 0, 2, :]
    drijkl[2, 3, 3, 1, :] = -drijkl[2, 3, 1, 3, :]
    drijkl[2, 3, 3, 2, :] = -drijkl[2, 3, 2, 3, :]

    #! --------------------------------------------------------------------------#

    drijkl[3, 0, 1, 2, :] = [0., M*a*(-3*(a**2*np.sin(theta)**2 - Delta)*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta*dSr - (a**2*np.sin(theta)**2 - Delta)*(a**2*np.cos(theta)**2 - 3.0*r**2)*Sigma*dD + (-6.0*r*(a**2*np.sin(theta)**2 - Delta) - (a**2*np.cos(theta)**2 - 3.0*r**2)*dD)*Delta*Sigma)/(Delta**2*Sigma**4*np.tan(theta)), M*a*(1.0*a**4*(1 - np.cos(2*theta))**2*Sigma + 2.5*a**4*Sigma*np.cos(2*theta) - 1.5*a**4*Sigma - 0.75*a**4*dSt*np.sin(2*theta) - 0.375*a**4*dSt*np.sin(4*theta) - 3.0*a**2*r**2*Sigma*np.cos(2*theta) + 4.5*a**2*r**2*dSt*np.sin(2*theta) + 1.0*a**2*Delta*Sigma*np.cos(2*theta) - 1.5*a**2*Delta*dSt*np.sin(2*theta) + 3.0*a**2*Delta*dSt/np.tan(theta) + 2.0*a**2*Delta*Sigma/(1 - np.cos(2*theta)) - 3.0*r**2*Delta*Sigma/np.sin(theta)**2 - 9.0*r**2*Delta*dSt/np.tan(theta))/(Delta*Sigma**4), 0.]

    drijkl[3, 1, 0, 1, :] = [0., M*a*(9.0*r*(3.0*a**2*np.cos(theta)**2 - r**2)*Delta*dSr + 3.0*r*(3.0*a**2*np.cos(theta)**2 - r**2)*Sigma*dD + 9.0*(-a**2*np.cos(theta)**2 + r**2)*Delta*Sigma)/(Delta**2*Sigma**4), 9.0*M*a*r*(a**2*Sigma*np.sin(2*theta) + (3.0*a**2*np.cos(theta)**2 - r**2)*dSt)/(Delta*Sigma**4), 0.]

    drijkl[3, 1, 0, 2, :] = [0., M*a*(-3*(2.0*a**2*np.sin(theta)**2 + Delta)*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta*dSr - (2.0*a**2*np.sin(theta)**2 + Delta)*(a**2*np.cos(theta)**2 - 3.0*r**2)*Sigma*dD + (-6.0*r*(2.0*a**2*np.sin(theta)**2 + Delta) + (a**2*np.cos(theta)**2 - 3.0*r**2)*dD)*Delta*Sigma)/(Delta**2*Sigma**4*np.tan(theta)), M*a*(2.0*a**4*(1 - np.cos(2*theta))**2*Sigma + 5.0*a**4*Sigma*np.cos(2*theta) - 3.0*a**4*Sigma - 1.5*a**4*dSt*np.sin(2*theta) - 0.75*a**4*dSt*np.sin(4*theta) - 6.0*a**2*r**2*Sigma*np.cos(2*theta) + 9.0*a**2*r**2*dSt*np.sin(2*theta) - 1.0*a**2*Delta*Sigma*np.cos(2*theta) + 1.5*a**2*Delta*dSt*np.sin(2*theta) - 3.0*a**2*Delta*dSt/np.tan(theta) - 2.0*a**2*Delta*Sigma/(1 - np.cos(2*theta)) + 3.0*r**2*Delta*Sigma/np.sin(theta)**2 + 9.0*r**2*Delta*dSt/np.tan(theta))/(Delta*Sigma**4), 0.]

    drijkl[3, 1, 1, 3, :] = [0., M*(3*r*(3.0*a**2*np.cos(theta)**2 - r**2)*(2.0*a**2*np.sin(theta)**2 + a**2 + r**2)*Delta*dSr + r*(3.0*a**2*np.cos(theta)**2 - r**2)*(2.0*a**2*np.sin(theta)**2 + a**2 + r**2)*Sigma*dD + (6.0*a**4*np.sin(theta)**4 - 3.0*a**4*np.sin(theta)**2 - 3.0*a**4 + 15.0*a**2*r**2*np.sin(theta)**2 - 6.0*a**2*r**2 + 5.0*r**4)*Delta*Sigma)/(Delta**2*Sigma**4), M*r*(a**2*(24.0*a**2*np.sin(theta)**2 - 6.0*a**2 + 10.0*r**2)*Sigma*np.sin(theta)*np.cos(theta) + 3*(3.0*a**2*np.cos(theta)**2 - r**2)*(2.0*a**2*np.sin(theta)**2 + a**2 + r**2)*dSt)/(Delta*Sigma**4), 0.]

    drijkl[3, 1, 2, 3, :] = [0., M*a**2*(-3*(a**2*np.cos(theta)**2 - 3.0*r**2)*(-2.0*M*r + 3.0*a**2 + 3.0*r**2)*Delta*dSr - (a**2*np.cos(theta)**2 - 3.0*r**2)*(-2.0*M*r + 3.0*a**2 + 3.0*r**2)*Sigma*dD + (-6.0*r*(-2.0*M*r + 3.0*a**2 + 3.0*r**2) + (-2.0*M + 6.0*r)*(a**2*np.cos(theta)**2 - 3.0*r**2))*Delta*Sigma)*np.sin(theta)*np.cos(theta)/(Delta**2*Sigma**4), M*a**2*(-3*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (4.0*a**2*np.sin(theta)**4 - 5.0*a**2*np.sin(theta)**2 + 1.0*a**2 + 6.0*r**2*np.sin(theta)**2 - 3.0*r**2)*Sigma)*(-2.0*M*r + 3.0*a**2 + 3.0*r**2)/(Delta*Sigma**4), 0.]

    drijkl[3, 2, 0, 1, :] = [0., M*a*(-3*(a**2*np.sin(theta)**2 + 2.0*Delta)*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta*dSr - (a**2*np.sin(theta)**2 + 2.0*Delta)*(a**2*np.cos(theta)**2 - 3.0*r**2)*Sigma*dD + (-6.0*r*(a**2*np.sin(theta)**2 + 2.0*Delta) + 2.0*(a**2*np.cos(theta)**2 - 3.0*r**2)*dD)*Delta*Sigma)/(Delta**2*Sigma**4*np.tan(theta)), M*a*(1.0*a**4*(1 - np.cos(2*theta))**2*Sigma + 2.5*a**4*Sigma*np.cos(2*theta) - 1.5*a**4*Sigma - 0.75*a**4*dSt*np.sin(2*theta) - 0.375*a**4*dSt*np.sin(4*theta) - 3.0*a**2*r**2*Sigma*np.cos(2*theta) + 4.5*a**2*r**2*dSt*np.sin(2*theta) - 2.0*a**2*Delta*Sigma*np.cos(2*theta) + 3.0*a**2*Delta*dSt*np.sin(2*theta) - 6.0*a**2*Delta*dSt/np.tan(theta) - 4.0*a**2*Delta*Sigma/(1 - np.cos(2*theta)) + 6.0*r**2*Delta*Sigma/np.sin(theta)**2 + 18.0*r**2*Delta*dSt/np.tan(theta))/(Delta*Sigma**4), 0.]

    drijkl[3, 2, 0, 2, :] = [0., 9.0*M*a*(-r*(3.0*a**2*np.cos(theta)**2 - r**2)*dSr + (a**2*np.cos(theta)**2 - r**2)*Sigma)/Sigma**4, -9.0*M*a*r*(a**2*Sigma*np.sin(2*theta) + (3.0*a**2*np.cos(theta)**2 - r**2)*dSt)/Sigma**4, 0.]

    drijkl[3, 2, 1, 3, :] = [0., M*a**2*(-3*(a**2*np.cos(theta)**2 - 3*r**2)*(-4.0*M*r + 3.0*a**2 + 3.0*r**2)*Delta*dSr - (a**2*np.cos(theta)**2 - 3*r**2)*(-4.0*M*r + 3.0*a**2 + 3.0*r**2)*Sigma*dD + (-6*r*(-4.0*M*r + 3.0*a**2 + 3.0*r**2) + (-4.0*M + 6.0*r)*(a**2*np.cos(theta)**2 - 3*r**2))*Delta*Sigma)*np.sin(theta)*np.cos(theta)/(Delta**2*Sigma**4), M*a**2*(-3*(a**2*np.cos(theta)**2 - 3*r**2)*dSt*np.sin(theta)*np.cos(theta) + (4*a**2*np.sin(theta)**4 - 5*a**2*np.sin(theta)**2 + a**2 + 6*r**2*np.sin(theta)**2 - 3*r**2)*Sigma)*(-4.0*M*r + 3.0*a**2 + 3.0*r**2)/(Delta*Sigma**4), 0.]

    drijkl[3, 2, 2, 3, :] = [0., M*(-3*r*(3.0*a**2*np.cos(theta)**2 - r**2)*(a**2*np.sin(theta)**2 + 2.0*a**2 + 2.0*r**2)*dSr + (-3.0*a**4*np.sin(theta)**4 - 3.0*a**4*np.sin(theta)**2 + 6.0*a**4 - 21.0*a**2*r**2*np.sin(theta)**2 + 12.0*a**2*r**2 - 10.0*r**4)*Sigma)/Sigma**4, M*r*(a**2*(12.0*a**2*np.cos(theta)**2 - 18.0*a**2 - 14.0*r**2)*Sigma*np.sin(theta)*np.cos(theta) - 3*(3.0*a**2*np.cos(theta)**2 - r**2)*(a**2*np.sin(theta)**2 + 2.0*a**2 + 2.0*r**2)*dSt)/Sigma**4, 0.] 

    drijkl[3, 3, 0, 3, :] = [0., M**2*a*r*(-8.0*r*(3.0*a**2*np.cos(theta)**2 - r**2)*dSr + 4.0*(3.0*a**2*np.cos(theta)**2 - 2.0*r**2)*Sigma)*np.sin(theta)**2/Sigma**5, M**2*a*r**2*(3.0*a**2*Sigma*np.sin(4*theta) + 3.0*a**2*dSt*np.cos(4*theta) - 3.0*a**2*dSt - 2.0*r**2*Sigma*np.sin(2*theta) - 4.0*r**2*dSt*np.cos(2*theta) + 4.0*r**2*dSt)/Sigma**5, 0.]

    drijkl[3, 3, 1, 2, :] = [0., M**2*a**2*(6.0*r*(a**2*np.cos(theta)**2 - 3.0*r**2)*Delta*dSr + 2.0*r*(a**2*np.cos(theta)**2 - 3.0*r**2)*Sigma*dD + (-2.0*a**2*np.cos(theta)**2 + 18.0*r**2)*Delta*Sigma)*np.sin(theta)*np.cos(theta)/(Delta**2*Sigma**4), M**2*a**2*r*(6.0*(a**2*np.cos(theta)**2 - 3.0*r**2)*dSt*np.sin(theta)*np.cos(theta) + (-8.0*a**2*np.sin(theta)**4 + 10.0*a**2*np.sin(theta)**2 - 2.0*a**2 - 12.0*r**2*np.sin(theta)**2 + 6.0*r**2)*Sigma)/(Delta*Sigma**4), 0.]

    # Symmetries
    drijkl[3, 0, 2, 1, :] = -drijkl[3, 0, 1, 2, :]
    drijkl[3, 1, 1, 0, :] = -drijkl[3, 1, 0, 1, :]
    drijkl[3, 1, 2, 0, :] = -drijkl[3, 1, 0, 2, :]
    drijkl[3, 1, 3, 1, :] = -drijkl[3, 1, 1, 3, :]
    drijkl[3, 1, 3, 2, :] = -drijkl[3, 1, 2, 3, :]
    drijkl[3, 2, 1, 0, :] = -drijkl[3, 2, 0, 1, :]
    drijkl[3, 2, 2, 0, :] = -drijkl[3, 2, 0, 2, :]
    drijkl[3, 2, 3, 1, :] = -drijkl[3, 2, 1, 3, :]
    drijkl[3, 2, 3, 2, :] = -drijkl[3, 2, 2, 3, :]
    drijkl[3, 3, 3, 0, :] = -drijkl[3, 3, 0, 3, :]
    drijkl[3, 3, 2, 1, :] = -drijkl[3, 3, 1, 2, :]

    #!---------------------------------------------------------------------#

    # part1 = a**2 + 2 * r * (-2 * M + r) + a**2 * sme.cos(2 * theta)
    # part2 = 3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta)
    # part3 = (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4
    # Calculating Rtensor elements

    drijkl[3, 0, 0, 3, :] = [0., M*(-8*r**2*(3*a**2*np.cos(theta)**2 - r**2)*(-2*M*r + a**2*np.cos(theta)**2 + r**2) + (a**2*np.cos(theta)**2 + r**2)*(-12*M*a**2*r*np.cos(theta)**2 + 8*M*r**3 + 3*a**4*(1 - np.cos(theta)**2)**2 + 6*a**4*np.cos(theta)**2 - 3*a**4 + 6*a**2*r**2*np.cos(theta)**2 - 5*r**4))/(a**2*np.cos(theta)**2 + r**2)**5, (1/2)*M*a**2*r*(-36*M*a**2*r*np.cos(theta)**2 + 28*M*r**3 + 12*a**4*(1 - np.cos(theta)**2)**2 + 24*a**4*np.cos(theta)**2 - 12*a**4 - 12*r**4)*np.sin(2*theta)/(a**2*np.cos(theta)**2 + r**2)**5, 0.]
    drijkl[3, 0, 3, 0, :] = -drijkl[3, 0, 0, 3, :]

    # return drijkl
    if config == "ullll":
        return drijkl + np.zeros((4, 4, 4, 4, 4))

    elif config == "lllll":
        gk = cov_metric_tensor(r=r, theta=theta, a=a, M=M)
        return np.einsum("ij,jklmp->iklmp", gk, drijkl) + np.zeros((4, 4, 4, 4, 4))

    # elif config=="lluu":
    #     #!Config: lluu
    #     gkinv = cot_metric_tensor(r=r, theta=theta, a=a, M=M)
    #     gk = cov_metric_tensor(r=r, theta=theta, a=a, M=M)
    #     rllll = np.einsum("ij,jklm->iklm", gk, drijkl)

    #   , :  return np.einsum("kjlm,la,mb->kjab", rllll, gkinv, gkinv)

    return drijkl

In [ ]:
def Dlevicevita(Dcov,guu, eps):
    dg= np.einsum("gd,gda->a",guu,Dcov)
    # return -0.5*eps*dg
    return -np.einsum("uvps,a->uvpsa",eps,dg)*0.5

In [ ]:
M, r, theta, a= 2.4, 43.49,np.pi/3.0, 3.2

In [ ]:
Dcov = Dcov_metric_tensor(r,theta,a,M)
guu= cot_metric_tensor(r, theta,a ,M)
gll= cov_metric_tensor(r, theta,a ,M)
eps= levi_civita_tensor(gll)

In [ ]:
def JacobianMPD(t, y, p):

    jacob= np.zeros((12,12),type(y[0]))

    t, r, theta, phi, pt, pr, ptheta, pphi, st, sr, stheta, sphi = y

    a, m, M = p

    # xvector=y[:4]
    pvector = y[4:8]  #! p_u
    svector = y[8:]  #! s_u

    gk = cov_metric_tensor(r, theta, a, M)  #! g_{ij}
    gkinv = cot_metric_tensor(r, theta, a, M)
    cs = kerr_christoffel(r, theta, a, M)  #! G^i_{jk}

    Riemann = kerr_riemann_tensor(r, theta, a, M, "ulll")  #! R^a_{bcd}

    # Riemann_cov = np.einsum("ul,lvps->uvps", gk, Riemann)  #! R_{abcd}
    # Riemann_uull = np.einsum("ap,upgd->uagd", gkinv, Riemann)  #! R^{ua}_{gd}

    levi = levi_civita_tensor(gk)  #! e^{abcd}
    # levi_mixed = np.einsum("psxy,xu,yv->psuv", levi, gk, gk)  #! e^{ps}_{uv}
    levi_mixed = np.einsum("px,sy,xyuv->psuv", gk, gk, levi)  #! e_{ps}^{uv}

    dRuluu = 0.5 * np.einsum("abps,psuv->abuv", Riemann, levi) #!R*^a_{buv}
    dRlluu = np.einsum("ia,abuv->ibuv", gk, dRuluu)  #!R*^_{ab}^{uv}

    ddRuuuu = 0.5 * np.einsum("bi,aips,psuv->abuv", gkinv, dRuluu, levi_mixed) #! *R*^{abuv}

    dgk= Dcov_metric_tensor(r,theta,a,M) #! Dgk_{auv}
    dgkinv= Dcot_metric_tensor(r, theta,a, M) #! Dgk^{uv}_a
    dcs= Dkerr_christoffel(r, theta,a ,M) #! Dcs^a_(bcd)
    dRiemann = DRijkl(r, theta, a, M, "ullll")  #! DR^a_{bcde}
    dg= np.einsum("gd,gda->a",gkinv,dgk) #! D(Det(g))
    dlevi = -np.einsum("uvps,a->uvpsa", levi, dg) * 0.5 #! Deps^{uvpsa}

    DdRuluu = 0.5 * (
        np.einsum("abpsk,psuv->abuvk", dRiemann, levi)
        + np.einsum("abps,psuvk->abuvk", Riemann, dlevi)
    )  #! DR*^a_{buv}

    DdRlluu = np.einsum("ai,ibupk->abupk", gk, DdRuluu)

    DddRuuuu = 0.5 * (
        np.einsum("bi,aipsk,psuv->abuvk", gkinv, DdRuluu, levi_mixed, optimize=True)
        + np.einsum(
            "bi,aips,pm,sn,mnuvz->abuvz", gkinv, ddRuuuu, gk, gk, dlevi, optimize=True
        )
    )  #! D*R*^{abuv}

    uvector = calculate_four_velocity(
        gk=gk, gkinv=gkinv, pvector=pvector, svector=svector, ddRuuuu=ddRuuuu
    )  #!dx_mu

    dp = calculate_four_momentum(
        pvector=pvector, uvector=uvector, svector=svector, cs=cs, dRlluu=dRlluu
    )
    #!dp_{mu}

    ds = calculate_four_spin(
        pvector=pvector, uvector=uvector, svector=svector, cs=cs, dRuluu=dRuluu
    )#! ds_mu

    wu= -np.einsum("uabg,a,b,g->u",ddRuuuu,svector,pvector,svector,optimize=True) #!w^mu

    N= 1/np.sqrt(1-2*np.einsum("u,u->",wu,pvector)-np.einsum("u,uj,j->",wu,gk,wu))

    puv= np.einsum("a,auv->uv",pvector,dgkinv) #! Dp^u_v

    wuv= -np.einsum("uabgv,a,b,g->uv",DddRuuuu,svector,pvector,svector,optimize=True) #! D w^m_v

    DN= N**3*(np.einsum("a,av->v",pvector,wuv)+ np.einsum("av,ag,g->v",wuv,gk,wu) +0.5*np.einsum("a,b,abv->v",wu,wu,dgk,optimize=True)) #! DN_v

    dx_dx= N*(puv+wuv+ np.einsum("u,az,z,av->uv",uvector,gk,uvector,wuv)+ np.einsum("u,a,b,abv->uv",uvector,wu,wu,dgk)*N/2) #! D \dot{x}/dx

    Wuv= -np.einsum("uavb,a,b->uv",ddRuuuu,svector,svector) #! W^{uv}

    dx_dP = N * (gkinv + Wuv + N * np.outer(uvector, wu)) + N * np.einsum(
        "u,ag,g,av->uv", uvector, gk, uvector, Wuv, optimize=True
    )  #! D \dot{x}/dp

    Vuv= -(np.einsum("a,b,uabv->uv",svector,pvector,ddRuuuu)- np.einsum("a,b,uvab->uv",svector,pvector,ddRuuuu)) #! V^{uv}

    dx_dS = N * Vuv + N * np.einsum(
        "u,az,z,av->uv", uvector, gk, uvector, Vuv
    )  #! D \dot{x}/dS

    term1= -np.einsum("a,b,ugabv,g->uv",pvector,svector,DdRlluu,uvector,optimize=True)
    term2 = -np.einsum(
        "a,b,ugab,gv->uv", pvector, svector, dRlluu, dx_dx, optimize=True
    )

    term3= np.einsum("a,abuv,b->uv",pvector,dcs,uvector)
    term4= np.einsum("a,abu,bv->uv",pvector,cs,dx_dx)

    dP_dx = term1 + term2 + term3 + term4

    term1= -np.einsum("b,ugvb,g->uv",svector,dRlluu,uvector)
    term2= -np.einsum("b,ugab,a,gv->uv",svector,dRlluu,pvector,dx_dP,optimize=True)
    term3= np.einsum("vbu,b->uv",cs,uvector) + np.einsum("abu,a,bv->uv",cs,pvector,dx_dP,optimize=True)

    dP_dP= term1+ term2+ term3

    term1= -np.einsum("ugav,g,a-> uv",dRlluu,uvector,pvector, optimize=True)
    term2= -np.einsum("ugab,gv,a,b->uv",dRlluu,dx_dS,pvector,svector,optimize=True)
    term3= np.einsum("abu,a,bv->uv",cs,pvector,dx_dS)

    dP_ds= term1+ term2+ term3

    term1= -np.einsum("u,a,g,d,abgdv,b->uv",pvector,svector,pvector,svector,DdRuluu,uvector,optimize=True)
    term2= -np.einsum("u,a,g,d,abgd,bv->uv",pvector,svector,pvector,svector,dRuluu,dx_dx,optimize=True)  
    term3= np.einsum("a,abuv,b->uv",svector,dcs,uvector) + np.einsum("a,abu,bv->uv",svector,cs,dx_dx,optimize=True)

    dS_dx= term1+ term2 + term3

    term1= -np.einsum("a,d,b,uv,abgd,g->uv",svector,svector,uvector,np.eye(4),dRuluu,pvector,optimize=True)
    term2= -np.einsum("a,d,b,u,abvd->uv",svector,svector,uvector,pvector,dRuluu,optimize=True)
    term3= -np.einsum("u,abgd,a,bv,g,d->uv",pvector,dRuluu,svector,dx_dP,pvector,svector,optimize=True)
    term4= np.einsum("abu,bv,a->uv",cs,dx_dP,svector)

    dS_dp= term1 +term3 + term2 + term4

    term1= -np.einsum("u,g,b,vbgd,d->uv",pvector,pvector,uvector,dRuluu,svector,optimize=True)
    term2= -np.einsum("u,g,b,abgv,a->uv",pvector,pvector,uvector,dRuluu,svector,optimize=True)
    term3= -np.einsum("u,abgd,a,bv,g,d->uv",pvector,dRuluu,svector,dx_dS,pvector,svector,optimize=True)
    term4= np.einsum("vbu,b->uv",cs,uvector)
    term5= np.einsum("abu,bv,a->uv",cs,dx_dS,svector,optimize=True)

    dS_dS= term1+ term2+ term3+ term4 + term5

    jacob[0:4,0:4]= dx_dx
    jacob[0:4,4:8]= dx_dP
    jacob[0:4,8:]= dx_dS

    jacob[4:8,0:4]= dP_dx
    jacob[4:8,4:8]= dP_dP
    jacob[4:8,8:]= dP_ds

    jacob[8:,0:4]= dS_dx
    jacob[8:,4:8]= dS_dp
    jacob[8:,8:]= dS_dS

    return jacob



In [ ]:
p

In [ ]:
y0.tolist()

In [ ]:
%%timeit
MPD(0.,y0,p)

In [ ]:
%%timeit
jacob= JacobianMPD(0.,y0,p)

In [ ]:
jacob[:,4]